![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/healthcare-nlp/08.0.Clinical_Text_Classification_with_SparkNLP.ipynb)

# Colab Setup

In [ ]:
! pip install -q johnsnowlabs

In [ ]:
from google.colab import files
print('Please Upload your John Snow Labs License using the button below')
license_keys = files.upload()

In [ ]:
from johnsnowlabs import nlp, medical

# After uploading your license run this to install all licensed Python Wheels and pre-download Jars the Spark Session JVM
nlp.install()

In [ ]:
from johnsnowlabs import nlp, medical, visual
import pandas as pd
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import pyspark.sql.functions as F

# Automatically load license data and start a session with all jars user has access to
spark = nlp.start(hardware_target="gpu")

👌 Detected license file /content/5.0.0.spark_nlp_for_healthcare.json
🤓 Looks like you are missing some jars, trying fetching them ...
👌 Detected license file /content/5.0.0.spark_nlp_for_healthcare.json
🙆 JSL Home setup in /root/.johnsnowlabs
👌 Detected license file /content/5.0.0.spark_nlp_for_healthcare.json
👌 Launched gpu optimized session with with: 🚀Spark-NLP==5.0.0, 💊Spark-Healthcare==5.0.0, running on ⚡ PySpark==3.1.2


# Classifiers

The below classifiers will be used in this notebook.ClassifierDL, MultiClassifierDL, and GenericClassifier will be trained using healthcare_100d, embeddingd_clinical, and bert sentence embeddings(sbiobert_base_cased_mli). DocumentLogRegClassifier accepts tokens, so sentence embeddings are not utilized during DocumentLogRegClassifier training.

## ClassifierDL

ClassifierDL is a generic Multi-class Text Classification annotator. ClassifierDL uses the state-of-the-art Universal Sentence Encoder as an input for text classifications. The ClassifierDL annotator uses a deep learning model (DNNs) built inside TensorFlow and supports up to 100 classes. For more information please [follow the link](https://nlp.johnsnowlabs.com/docs/en/annotators#classifierdl).


##  MultiClassifierDL

MultiClassifierDL is a Multi-label Text Classification annotator.MultiClassifierDL uses a Bidirectional GRU with a convolutional model built inside TensorFlow and supports up to 100 classes.  For more information please [follow the link](https://nlp.johnsnowlabs.com/docs/en/annotators#multiclassifierdl).


## GenericClassifier

GenericClassifier is a TensorFlow model for the generic classification of feature vectors in Healthcare  Lİbrary. It takes FEATURE_VECTOR annotations from FeaturesAssembler as input, classifies them, and outputs CATEGORY annotations. Please see [the link](https://nlp.johnsnowlabs.com/docs/en/licensed_annotators#genericclassifier) for more information.

Here are some Social Determinants of Health (SDOH) models that trained with GenericClassifier:

|index|model|
|-----:|:-----|
|1|[genericclassifier_sdoh_economics_binary_sbiobert_cased_mli](https://nlp.johnsnowlabs.com/2023/01/14/genericclassifier_sdoh_economics_binary_sbiobert_cased_mli_en.html)|
|2|[genericclassifier_sdoh_substance_usage_binary_sbiobert_cased_mli](https://nlp.johnsnowlabs.com/2023/01/14/genericclassifier_sdoh_substance_usage_binary_sbiobert_cased_mli_en.html)|
|3|[genericclassifier_sdoh_tobacco_usage_sbiobert_cased_mli](https://nlp.johnsnowlabs.com/2023/01/14/genericclassifier_sdoh_tobacco_usage_sbiobert_cased_mli_en.html)|
|4|[genericclassifier_sdoh_alcohol_usage_sbiobert_cased_mli](https://nlp.johnsnowlabs.com/2023/01/14/genericclassifier_sdoh_alcohol_usage_sbiobert_cased_mli_en.html)|
|5|[genericclassifier_sdoh_alcohol_usage_binary_sbiobert_cased_mli](https://nlp.johnsnowlabs.com/2023/01/14/genericclassifier_sdoh_alcohol_usage_binary_sbiobert_cased_mli_en.html)|


## GenericLogRegClassifier

`GenericLogRegClassifier` is a derivative of GenericClassifier which implements a multinomial *Logistic Regression*. This is a single layer neural network with the logistic function at the output. The input to the model is `FeatureVector` and the output is `Category` annotations with labels and corresponding confidence scores varying between 0 and 1. Training data requires "text" and their "label" columns only and the trained model will be a `GenericLogRegClassifierModel()`.


## GenericSVMClassifier

`GenericSVMClassifier` is a derivative of GenericClassifier which implements *SVM (Support Vector Machine)* classification. The input to the model is `FeatureVector` and the output is `Category` annotations with labels and corresponding confidence scores. The scores are standardized using the logistic function so that they vary between 0 and 1. Taining data requires "text" and their "label" columns only and the trained model will be a `GenericSVMClassifierModel()`

## DocumentLogRegClassifier

DocumentLogRegClassifier is a model to classify documents with a Logarithmic Regression algorithm in Healthcare  Library. Training data requires columns for text and labels. The result is a trained DocumentLogRegClassifierModel. you can get more info [here](https://nlp.johnsnowlabs.com/docs/en/licensed_annotators#documentlogregclassifier).


# ADE Dataset

### Data Preprocessing

In [ ]:
#downloading sample datasets
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/data/ADE_Corpus_V2/ADE-NEG.txt
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/data/ADE_Corpus_V2/DRUG-AE.rel

**ADE Negative Dataset**

In [ ]:
df_neg= pd.read_csv("ADE-NEG.txt", header=None, delimiter="\t", names=["col1"])
df_neg.head()

,col1
0,6460590 NEG Clioquinol intoxication occurring ...
1,"8600337 NEG ""Retinoic acid syndrome"" was preve..."
2,8402502 NEG BACKGROUND: External beam radiatio...
3,"8700794 NEG Although the enuresis ceased, she ..."
4,17662448 NEG A 42-year-old woman had uneventfu...


In [ ]:
df_neg['text'] =  df_neg.col1.str.split('NEG').str[1]
df_neg["category"] = "neg"
df_neg= df_neg[["text", "category"]]
df_neg.head()

,text,category
0,Clioquinol intoxication occurring in the trea...,neg
1,"""Retinoic acid syndrome"" was prevented with s...",neg
2,BACKGROUND: External beam radiation therapy o...,neg
3,"Although the enuresis ceased, she developed t...",neg
4,A 42-year-old woman had uneventful bilateral ...,neg


**ADE Positive Dataset**

In [ ]:
df_pos= pd.read_csv("DRUG-AE.rel", header=None, delimiter="|")
df_pos.head()

,0,1,2,3,4,5,6,7
0,10030778,Intravenous azithromycin-induced ototoxicity.,ototoxicity,43,54,azithromycin,22,34
1,10048291,"Immobilization, while Paget's bone disease was...",increased calcium-release,960,985,dihydrotachysterol,908,926
2,10048291,Unaccountable severe hypercalcemia in a patien...,hypercalcemia,31,44,dihydrotachysterol,94,112
3,10082597,METHODS: We report two cases of pseudoporphyri...,pseudoporphyria,620,635,naproxen,646,654
4,10082597,METHODS: We report two cases of pseudoporphyri...,pseudoporphyria,620,635,oxaprozin,659,668


In [ ]:
df_pos["category"]= "pos"
df_pos.rename(columns={1: "text"}, inplace=True)
df_pos= df_pos[["text", "category"]]
df_pos.head()

,text,category
0,Intravenous azithromycin-induced ototoxicity.,pos
1,"Immobilization, while Paget's bone disease was...",pos
2,Unaccountable severe hypercalcemia in a patien...,pos
3,METHODS: We report two cases of pseudoporphyri...,pos
4,METHODS: We report two cases of pseudoporphyri...,pos


**Merging Positive and Negative dataset**

In [ ]:
ade_df= pd.concat([df_neg, df_pos])
ade_df.head()

,text,category
0,Clioquinol intoxication occurring in the trea...,neg
1,"""Retinoic acid syndrome"" was prevented with s...",neg
2,BACKGROUND: External beam radiation therapy o...,neg
3,"Although the enuresis ceased, she developed t...",neg
4,A 42-year-old woman had uneventful bilateral ...,neg


In [ ]:
ade_df["category"].value_counts()

neg    16695
pos     6821
Name: category, dtype: int64

In [ ]:
ade_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23516 entries, 0 to 6820
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   text      23516 non-null  object
 1   category  23516 non-null  object
dtypes: object(2)
memory usage: 551.2+ KB


We take 30% of the data to make a faster run. You can use all data for better scores.

In [ ]:
spark_df = spark.createDataFrame(ade_df).sample(0.3, 3) # limit the data

trainingData, testData = spark_df.randomSplit([0.8, 0.2], seed = 100)

print("Training Dataset Count: " + str(trainingData.count()))
print("Test Dataset Count: " + str(testData.count()))

Training Dataset Count: 5552
Test Dataset Count: 1454


In [ ]:
spark_df.groupBy("category").count().show()

+--------+-----+
|category|count|
+--------+-----+
|     pos| 2002|
|     neg| 5004|
+--------+-----+



In [ ]:
spark_df.printSchema()

root
 |-- text: string (nullable = true)
 |-- category: string (nullable = true)



In [ ]:
spark_df.head(3)

[Row(text=' Clioquinol intoxication occurring in the treatment of acrodermatitis enteropathica with reference to SMON outside of Japan.', category='neg'),
 Row(text=' A 42-year-old woman had uneventful bilateral laser-assisted subepithelial keratectomy (LASEK) to correct myopia.', category='neg'),
 Row(text=' A 16-year-old girl with erosive, polyarticular JRA showed no detectable change in her articular disease following nine exchanges.', category='neg')]

## 100 Dimension Healthcare Embeddings (embeddings_healthcare_100d)



Now we will extract [healthcare_100d embeddings](https://nlp.johnsnowlabs.com/2020/05/29/embeddings_healthcare_100d_en.html) and use it in the classificaiton model training.

In [ ]:
document_assembler = nlp.DocumentAssembler()\
        .setInputCol("text")\
        .setOutputCol("document")

tokenizer = nlp.Tokenizer() \
            .setInputCols(["document"]) \
            .setOutputCol("token")

word_embeddings = nlp.WordEmbeddingsModel.pretrained("embeddings_healthcare_100d","en","clinical/models")\
        .setInputCols(["document","token"])\
        .setOutputCol("word_embeddings")

sentence_embeddings = nlp.SentenceEmbeddings() \
        .setInputCols(["document", "word_embeddings"]) \
        .setOutputCol("sentence_embeddings") \
        .setPoolingStrategy("AVERAGE")

embeddings_pipeline = nlp.Pipeline(
    stages = [
        document_assembler,
        tokenizer,
        word_embeddings,
        sentence_embeddings,

    ])

embeddings_healthcare_100d download started this may take some time.
Approximate size to download 475.8 MB
[OK!]


In [ ]:
trainingData_with_embeddings = embeddings_pipeline.fit(trainingData).transform(trainingData)
trainingData_with_embeddings = trainingData_with_embeddings.select("text","category","sentence_embeddings")
trainingData_with_embeddings.show(2,truncate=60)

+------------------------------------------------------------+--------+------------------------------------------------------------+
|                                                        text|category|                                         sentence_embeddings|
+------------------------------------------------------------+--------+------------------------------------------------------------+
| (4) The acute rehabilitation of a 21-year-old woman with...|     neg|[{sentence_embeddings, 0, 182,  (4) The acute rehabilitat...|
| A 16-year-old girl with erosive, polyarticular JRA showe...|     neg|[{sentence_embeddings, 0, 129,  A 16-year-old girl with e...|
+------------------------------------------------------------+--------+------------------------------------------------------------+
only showing top 2 rows



In [ ]:
testData_with_embeddings = embeddings_pipeline.fit(testData).transform(testData)
testData_with_embeddings = testData_with_embeddings.select("text","category","sentence_embeddings")
testData_with_embeddings.show(2,truncate=60)

+------------------------------------------------------------+--------+------------------------------------------------------------+
|                                                        text|category|                                         sentence_embeddings|
+------------------------------------------------------------+--------+------------------------------------------------------------+
| A 51-year-old-woman presented with chronic eosinophilia,...|     neg|[{sentence_embeddings, 0, 139,  A 51-year-old-woman prese...|
| Additionally, their Global Assessment Scale (GAS) score ...|     neg|[{sentence_embeddings, 0, 164,  Additionally, their Globa...|
+------------------------------------------------------------+--------+------------------------------------------------------------+
only showing top 2 rows



In [ ]:
testData_with_embeddings.printSchema()

root
 |-- text: string (nullable = true)
 |-- category: string (nullable = true)
 |-- sentence_embeddings: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)



In [ ]:
testData_with_embeddings.select(testData_with_embeddings.sentence_embeddings.embeddings).show(3,truncate=120)

+------------------------------------------------------------------------------------------------------------------------+
|                                                                                          sentence_embeddings.embeddings|
+------------------------------------------------------------------------------------------------------------------------+
|[[0.04241796, 0.10478715, 0.042885326, 0.06311753, -0.3185922, 0.046458624, 0.17697075, -0.037831675, -0.01754189, -0...|
|[[-0.051357962, -0.014281397, 0.048104603, 0.09546186, -0.115982704, -0.02890453, 0.0554792, -0.09107009, -0.01669618...|
|[[0.023146994, -0.048840538, 0.2768485, 0.09796163, -0.13117273, -0.0791464, 0.068399966, -0.04242571, -0.025790978, ...|
+------------------------------------------------------------------------------------------------------------------------+
only showing top 3 rows



In [ ]:
log_folder="ADE_logs_healthcare_100d"
!mkdir -p $log_folder

### ClassifierDL

In [ ]:
classifier_dl = nlp.ClassifierDLApproach()\
        .setInputCols(["sentence_embeddings"])\
        .setOutputCol("prediction")\
        .setLabelColumn("category")\
        .setBatchSize(16)\
        .setMaxEpochs(30)\
        .setLr(0.002)\
        .setDropout(0.3)\
        .setEnableOutputLogs(True)\
        .setOutputLogsPath(log_folder)\
        # .setValidationSplit(0.1)

classifier_dl_pipeline = nlp.Pipeline(
    stages = [
        classifier_dl
    ])

In [ ]:
clfDL_model_hc100 = classifier_dl_pipeline.fit(trainingData_with_embeddings)

In [ ]:
!cat $log_folder/ClassifierDLApproach_*

Training started - epochs: 30 - learning_rate: 0.002 - batch_size: 16 - training_examples: 5552 - classes: 2
Epoch 0/30 - 1.64s - loss: 180.21819 - acc: 0.75306195 - batches: 347
Epoch 1/30 - 1.09s - loss: 164.6449 - acc: 0.8058357 - batches: 347
Epoch 2/30 - 1.06s - loss: 159.58823 - acc: 0.8252882 - batches: 347
Epoch 3/30 - 1.05s - loss: 154.76198 - acc: 0.83753604 - batches: 347
Epoch 4/30 - 1.05s - loss: 151.47258 - acc: 0.84762245 - batches: 347
Epoch 5/30 - 1.05s - loss: 149.46481 - acc: 0.85320604 - batches: 347
Epoch 6/30 - 1.04s - loss: 147.91342 - acc: 0.8564481 - batches: 347
Epoch 7/30 - 1.03s - loss: 146.76279 - acc: 0.85933 - batches: 347
Epoch 8/30 - 1.09s - loss: 145.72246 - acc: 0.8631124 - batches: 347
Epoch 9/30 - 1.05s - loss: 144.8968 - acc: 0.86689484 - batches: 347
Epoch 10/30 - 1.05s - loss: 144.20969 - acc: 0.87067723 - batches: 347
Epoch 11/30 - 1.00s - loss: 143.65863 - acc: 0.87355906 - batches: 347
Epoch 12/30 - 1.02s - loss: 143.15186 - acc: 0.87698126 - 

In [ ]:
preds = clfDL_model_hc100.transform(testData_with_embeddings).cache()

In [ ]:
preds.printSchema()
preds.select(preds.prediction).show(5, truncate=False)
preds.select(preds.category, preds.prediction.result).show(5, truncate=False)

root
 |-- text: string (nullable = true)
 |-- category: string (nullable = true)
 |-- sentence_embeddings: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)
 |-- prediction: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    

In [ ]:
preds_df = preds.select("category","text","prediction.result").toPandas()
preds_df["result"] = preds_df["result"].apply(lambda x : x[0])

print (classification_report(preds_df["category"], preds_df["result"]))

              precision    recall  f1-score   support

         neg       0.85      0.91      0.88      1031
         pos       0.74      0.60      0.66       423

    accuracy                           0.82      1454
   macro avg       0.79      0.76      0.77      1454
weighted avg       0.82      0.82      0.82      1454



In [ ]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df = pd.DataFrame(columns=["pos-f1-score","accuracy"])
results_df.loc["ClassifierDL_100d"] = [res["pos"]["f1-score"], res["accuracy"]]
results_df

,pos-f1-score,accuracy
ClassifierDL_100d,0.661458,0.821183


### MultiClassifierDL

We will use MultiClassifierDL built by using Bidirectional GRU and CNNs inside TensorFlow that supports up to 100 classes. It is designed for multi-label classification purposes. Here we will use MultiClassifierDL as a binary classifier.

In [ ]:
# MultiClassifierDL accepts list of strings as input label. So we convert label column to array type.
trainingData_with_embeddings = trainingData_with_embeddings.withColumn("category_array", F.array(F.col("category")))
testData_with_embeddings = testData_with_embeddings.withColumn("category_array", F.array(F.col("category")))

In [ ]:
multiClassifier = nlp.MultiClassifierDLApproach()\
  .setInputCols("sentence_embeddings")\
  .setOutputCol("prediction")\
  .setLabelColumn("category_array")\
  .setBatchSize(16)\
  .setMaxEpochs(20)\
  .setLr(9e-3)\
  .setThreshold(0.5)\
  .setShufflePerEpoch(False)\
  .setEnableOutputLogs(True)\
  .setOutputLogsPath(log_folder)\
#   .setValidationSplit(0.1)

multiClassifier_pipeline = nlp.Pipeline(
    stages = [
        multiClassifier
    ])

In [ ]:
multiClassifier_model_hc100 = multiClassifier_pipeline.fit(trainingData_with_embeddings)

In [ ]:
!cat $log_folder/MultiClassifierDLApproach_*

Training started - epochs: 20 - learning_rate: 0.009 - batch_size: 16 - training_examples: 5552 - classes: 2
Epoch 0/20 - 6.86s - loss: 0.51096046 - acc: 0.7431556 - batches: 347
Epoch 1/20 - 2.07s - loss: 0.44683927 - acc: 0.7823307 - batches: 347
Epoch 2/20 - 2.18s - loss: 0.41658133 - acc: 0.8006124 - batches: 347
Epoch 3/20 - 2.16s - loss: 0.39496046 - acc: 0.8125 - batches: 347
Epoch 4/20 - 2.16s - loss: 0.37632382 - acc: 0.82582855 - batches: 347
Epoch 5/20 - 2.17s - loss: 0.36035827 - acc: 0.8337536 - batches: 347
Epoch 6/20 - 2.30s - loss: 0.34597775 - acc: 0.8434798 - batches: 347
Epoch 7/20 - 2.22s - loss: 0.33060083 - acc: 0.84789264 - batches: 347
Epoch 8/20 - 2.19s - loss: 0.31673864 - acc: 0.8573487 - batches: 347
Epoch 9/20 - 2.13s - loss: 0.30340043 - acc: 0.86374277 - batches: 347
Epoch 10/20 - 2.07s - loss: 0.2889029 - acc: 0.87193805 - batches: 347
Epoch 11/20 - 2.07s - loss: 0.2746806 - acc: 0.8803134 - batches: 347
Epoch 12/20 - 2.05s - loss: 0.26032013 - acc: 0.88

In [ ]:
preds = multiClassifier_model_hc100.transform(testData_with_embeddings).cache()

In [ ]:
preds.printSchema()
preds.select(preds.prediction).show(5, truncate=False)
preds.select(preds.category_array, preds.prediction.result).show(5, truncate=False)

root
 |-- text: string (nullable = true)
 |-- category: string (nullable = true)
 |-- sentence_embeddings: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)
 |-- category_array: array (nullable = false)
 |    |-- element: string (containsNull = true)
 |-- prediction: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable

In [ ]:
preds_df = preds.select("category","text","prediction.result","prediction.metadata").toPandas()
preds_df.result.apply(lambda x: str(x) ).value_counts()

['neg']    1021
['pos']     432
[]            1
Name: result, dtype: int64

In [ ]:
preds_df[preds_df.result.apply(len)==2]

,category,text,result,metadata


In [ ]:
preds_df[preds_df.result.apply(len)==0]

,category,text,result,metadata
365,neg,"However, five patients showed delayed 30-Hz f...",[],[]


MultiClassifierDL is a multi-label classifier, so some predictions may include both labels or none of the labels. That can be controlled a bit with `.setThreshold()` parameter during training. For now we will not keep zero label predictions and get the highest score as prediction.

In [ ]:
# We will get the highest score label as result. You can control the number of zero label results with setThreshold() in the training.
preds_df["scores"] = preds_df.metadata.apply(lambda x: {k:float(v) for k,v in x[0].items()} if len(x)>=1 else "")
preds_df["result"] = preds_df.scores.apply(lambda x: max(x, key=x.get) if len(x)>=1 else "")
preds_df = preds_df[(preds_df["result"]!="")]
preds_df["result"].value_counts()

neg    1021
pos     432
Name: result, dtype: int64

In [ ]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["MultiClassifierDL_100d"] = [res["pos"]["f1-score"], res["accuracy"]]

print (classification_report(preds_df["category"], preds_df["result"]))

              precision    recall  f1-score   support

         neg       0.86      0.85      0.86      1030
         pos       0.65      0.66      0.65       423

    accuracy                           0.80      1453
   macro avg       0.75      0.76      0.76      1453
weighted avg       0.80      0.80      0.80      1453



### Generic Classifier

In [ ]:
!pip install -q tensorflow==2.11.0 tensorflow_addons

In [ ]:
from sparknlp_jsl.annotator import TFGraphBuilder

graph_folder = "gc_graph"

gc_graph_builder =medical.TFGraphBuilder()\
    .setModelName("generic_classifier")\
    .setInputCols(["features"])\
    .setLabelColumn("category")\
    .setHiddenLayers([300,200, 50])\
    .setHiddenAct("tanh")\
    .setHiddenActL2(True)\
    .setBatchNorm(True)\
    .setGraphFolder(graph_folder)\
    .setGraphFile("gcf_graph_100d.pb")

GenericClassifier needs outputs from FeaturesAssembler. The FeaturesAssembler is used to collect features from different columns or an embeddings column.

The GenericClassifierApproach takes FEATURE_VECTOR annotations as input, classifies them and outputs CATEGORY annotations.

In [ ]:
features_asm = medical.FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("features")

gen_clf =medical.GenericClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("features")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/gcf_graph_100d.pb")\
    .setEpochsNumber(25)\
    .setBatchSize(128)\
    .setLearningRate(0.004)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.2)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = nlp.Pipeline(stages=[
    features_asm,
    gc_graph_builder,
    gen_clf])


In [ ]:
generic_model_hc100 = clf_Pipeline.fit(trainingData_with_embeddings)

TF Graph Builder configuration:
Model name: generic_classifier
Graph folder: gc_graph
Graph file name: gcf_graph_100d.pb
Build params: {'input_dim': 100, 'output_dim': 2, 'hidden_layers': [300, 200, 50], 'hidden_act': 'tanh', 'hidden_act_l2': True, 'batch_norm': True}


Instructions for updating:
Colocations handled automatically by placer.


generic_classifier graph exported to gc_graph/gcf_graph_100d.pb


In [ ]:
!cat $log_folder/GenericClassifierApproach_*

Training 25 epochs
Epoch 1/25	0.35s	Loss: 10.994606	ACC: 0.6780895
Epoch 2/25	0.12s	Loss: 9.687545	ACC: 0.7283973
Epoch 3/25	0.12s	Loss: 9.226147	ACC: 0.7372751
Epoch 4/25	0.12s	Loss: 9.271717	ACC: 0.7331913
Epoch 5/25	0.12s	Loss: 9.069597	ACC: 0.7425426
Epoch 6/25	0.12s	Loss: 8.769637	ACC: 0.7527817
Epoch 7/25	0.12s	Loss: 8.602073	ACC: 0.7581084
Epoch 8/25	0.12s	Loss: 8.2566185	ACC: 0.77101094
Epoch 9/25	0.12s	Loss: 8.825014	ACC: 0.75852275
Epoch 10/25	0.12s	Loss: 8.394891	ACC: 0.7693537
Epoch 11/25	0.11s	Loss: 8.092277	ACC: 0.7843868
Epoch 12/25	0.12s	Loss: 8.261728	ACC: 0.77533144
Epoch 13/25	0.12s	Loss: 8.129594	ACC: 0.77840906
Epoch 14/25	0.12s	Loss: 8.1621685	ACC: 0.77681106
Epoch 15/25	0.12s	Loss: 8.083638	ACC: 0.7797703
Epoch 16/25	0.12s	Loss: 8.114319	ACC: 0.77509475
Epoch 17/25	0.12s	Loss: 7.9031367	ACC: 0.79231775
Epoch 18/25	0.12s	Loss: 7.820131	ACC: 0.79178506
Epoch 19/25	0.12s	Loss: 7.6165056	ACC: 0.795277
Epoch 20/25	0.12s	Loss: 8.1274605	ACC: 0.78420925
Epoch 21/25	0.12

In [ ]:
pred_df = generic_model_hc100.transform(testData_with_embeddings).cache()

In [ ]:
preds.printSchema()
preds.select(preds.prediction).show(5, truncate=False)
preds.select(preds.category, preds.prediction.result).show(5, truncate=False)

root
 |-- text: string (nullable = true)
 |-- category: string (nullable = true)
 |-- sentence_embeddings: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)
 |-- category_array: array (nullable = false)
 |    |-- element: string (containsNull = true)
 |-- prediction: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable

In [ ]:
preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

              precision    recall  f1-score   support

         neg       0.88      0.78      0.83      1031
         pos       0.58      0.74      0.65       423

    accuracy                           0.77      1454
   macro avg       0.73      0.76      0.74      1454
weighted avg       0.79      0.77      0.78      1454



In [ ]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["GenericClassifier_100d"] = [res["pos"]["f1-score"], res["accuracy"]]

### GenericLogRegClassifier

In [ ]:
graph_folder = "gc_graph"

gc_logreg_graph_builder = medical.TFGraphBuilder()\
    .setModelName("logreg_classifier")\
    .setInputCols(["feature_vector"]) \
    .setLabelColumn("category")\
    .setGraphFolder(graph_folder)\
    .setGraphFile("log_reg_graph.pb")

GenericLogRegClassifier needs outputs from FeaturesAssembler. The FeaturesAssembler is used to collect features from different columns or an embeddings column.

The GenericLogRegClassifier takes FEATURE_VECTOR annotations as input, classifies them and outputs CATEGORY annotations.

In [ ]:
features_asm = medical.FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("feature_vector")

gen_clf = medical.GenericLogRegClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("feature_vector")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/log_reg_graph.pb")\
    .setEpochsNumber(20)\
    .setBatchSize(128)\
    .setLearningRate(0.01)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.1)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = nlp.Pipeline(stages=[
    features_asm,
    gc_logreg_graph_builder,
    gen_clf])


In [ ]:
generic_model_hc100 = clf_Pipeline.fit(trainingData_with_embeddings)

TF Graph Builder configuration:
Model name: logreg_classifier
Graph folder: gc_graph
Graph file name: log_reg_graph.pb
Build params: {'input_dim': 100, 'output_dim': 2, 'hidden_layers': [], 'output_act': 'sigmoid'}
logreg_classifier graph exported to gc_graph/log_reg_graph.pb


In [ ]:
!cat $log_folder/GenericLogRegClassifierApproach*

Training 20 epochs
Epoch 1/20	0.12s	Loss: 26.28012	ACC: 0.70170456
Epoch 2/20	0.04s	Loss: 24.171764	ACC: 0.72419506
Epoch 3/20	0.04s	Loss: 23.208557	ACC: 0.73325044
Epoch 4/20	0.04s	Loss: 22.77773	ACC: 0.7401752
Epoch 5/20	0.04s	Loss: 22.327833	ACC: 0.7455019
Epoch 6/20	0.04s	Loss: 22.129986	ACC: 0.74822444
Epoch 7/20	0.04s	Loss: 21.910912	ACC: 0.74923056
Epoch 8/20	0.04s	Loss: 21.59937	ACC: 0.75639206
Epoch 9/20	0.04s	Loss: 21.4923	ACC: 0.75485325
Epoch 10/20	0.04s	Loss: 21.411522	ACC: 0.7578125
Epoch 11/20	0.04s	Loss: 21.390123	ACC: 0.7563328
Epoch 12/20	0.04s	Loss: 21.261513	ACC: 0.7587003
Epoch 13/20	0.04s	Loss: 21.270958	ACC: 0.7592922
Epoch 14/20	0.04s	Loss: 21.081085	ACC: 0.7652699
Epoch 15/20	0.04s	Loss: 21.073338	ACC: 0.7643229
Epoch 16/20	0.04s	Loss: 20.96593	ACC: 0.7598248
Epoch 17/20	0.04s	Loss: 21.088984	ACC: 0.76160043
Epoch 18/20	0.04s	Loss: 21.042347	ACC: 0.7590554
Epoch 19/20	0.04s	Loss: 20.728872	ACC: 0.766572
Epoch 20/20	0.04s	Loss: 20.89069	ACC: 0.766276
Training co

In [ ]:
pred_df = generic_model_hc100.transform(testData_with_embeddings)

In [ ]:
preds.printSchema()
preds.select(preds.prediction).show(5, truncate=False)
preds.select(preds.category, preds.prediction.result).show(5, truncate=False)

root
 |-- text: string (nullable = true)
 |-- category: string (nullable = true)
 |-- sentence_embeddings: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)
 |-- category_array: array (nullable = false)
 |    |-- element: string (containsNull = true)
 |-- prediction: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable

In [ ]:
preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

              precision    recall  f1-score   support

         neg       0.77      0.93      0.84      1031
         pos       0.66      0.34      0.45       423

    accuracy                           0.76      1454
   macro avg       0.72      0.64      0.65      1454
weighted avg       0.74      0.76      0.73      1454



In [ ]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["GenericLogReg_100d"] = [res["pos"]["f1-score"], res["accuracy"]]

### GenericSVMClassifier

In [ ]:
graph_folder = "gc_graph"

gc_svm_graph_builder = medical.TFGraphBuilder()\
    .setModelName("svm_classifier")\
    .setInputCols(["feature_vector"]) \
    .setLabelColumn("category")\
    .setGraphFolder(graph_folder)\
    .setGraphFile("svm_graph.pb")

GenericSVMClassifier needs outputs from FeaturesAssembler. The FeaturesAssembler is used to collect features from different columns or an embeddings column.

The GenericSVMClassifier takes FEATURE_VECTOR annotations as input, classifies them and outputs CATEGORY annotations.

In [ ]:
features_asm = medical.FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("feature_vector")

gen_clf = medical.GenericSVMClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("feature_vector")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/svm_graph.pb")\
    .setEpochsNumber(25)\
    .setBatchSize(128)\
    .setLearningRate(0.015)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.1)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = nlp.Pipeline(stages=[
    features_asm,
    gc_svm_graph_builder,
    gen_clf])


In [ ]:
generic_model_hc100 = clf_Pipeline.fit(trainingData_with_embeddings)

TF Graph Builder configuration:
Model name: svm_classifier
Graph folder: gc_graph
Graph file name: svm_graph.pb
Build params: {'input_dim': 100, 'output_dim': 2, 'hidden_layers': [], 'output_act': 'sigmoid', 'loss_func': 'hinge'}
svm_classifier graph exported to gc_graph/svm_graph.pb


In [ ]:
!cat $log_folder/GenericSVMClassifierApproach*

Training 25 epochs
Epoch 1/25	0.13s	Loss: 28.263968	ACC: 0.70963544
Epoch 2/25	0.04s	Loss: 24.892042	ACC: 0.72585225
Epoch 3/25	0.04s	Loss: 24.05352	ACC: 0.7391098
Epoch 4/25	0.04s	Loss: 23.965277	ACC: 0.7421283
Epoch 5/25	0.04s	Loss: 23.761341	ACC: 0.7452652
Epoch 6/25	0.04s	Loss: 23.8627	ACC: 0.750947
Epoch 7/25	0.04s	Loss: 23.443958	ACC: 0.75378793
Epoch 8/25	0.04s	Loss: 23.236681	ACC: 0.75710225
Epoch 9/25	0.04s	Loss: 23.368925	ACC: 0.7589962
Epoch 10/25	0.04s	Loss: 23.538113	ACC: 0.75710225
Epoch 11/25	0.04s	Loss: 23.098625	ACC: 0.7619555
Epoch 12/25	0.04s	Loss: 23.590431	ACC: 0.75946975
Epoch 13/25	0.04s	Loss: 23.300802	ACC: 0.7614229
Epoch 14/25	0.04s	Loss: 23.38547	ACC: 0.762429
Epoch 15/25	0.04s	Loss: 23.57997	ACC: 0.75946975
Epoch 16/25	0.04s	Loss: 23.164564	ACC: 0.7619555
Epoch 17/25	0.04s	Loss: 23.252136	ACC: 0.7659209
Epoch 18/25	0.04s	Loss: 23.289877	ACC: 0.76165956
Epoch 19/25	0.04s	Loss: 23.232693	ACC: 0.76266575
Epoch 20/25	0.04s	Loss: 23.816528	ACC: 0.76171875
Epoch 2

In [ ]:
pred_df = generic_model_hc100.transform(testData_with_embeddings)

In [ ]:
preds.printSchema()
preds.select(preds.prediction).show(5, truncate=False)
preds.select(preds.category, preds.prediction.result).show(5, truncate=False)

root
 |-- text: string (nullable = true)
 |-- category: string (nullable = true)
 |-- sentence_embeddings: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)
 |-- category_array: array (nullable = false)
 |    |-- element: string (containsNull = true)
 |-- prediction: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable

In [ ]:
preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

              precision    recall  f1-score   support

         neg       0.77      0.94      0.85      1031
         pos       0.68      0.32      0.43       423

    accuracy                           0.76      1454
   macro avg       0.73      0.63      0.64      1454
weighted avg       0.74      0.76      0.73      1454



In [ ]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["GenericSVM_100d"] = [res["pos"]["f1-score"], res["accuracy"]]

## 200 Dimension Healthcare Embeddings (embeddings_clinical)



Now we will extract [embeddings_clinical](https://nlp.johnsnowlabs.com/2020/01/28/embeddings_clinical_en.html) embeddings which has 200 dimension output and use this embeddings in the model training.

In [ ]:
document_assembler = nlp.DocumentAssembler()\
        .setInputCol("text")\
        .setOutputCol("document")

tokenizer = nlp.Tokenizer() \
            .setInputCols(["document"]) \
            .setOutputCol("token")

word_embeddings = nlp.WordEmbeddingsModel.pretrained("embeddings_clinical","en","clinical/models")\
        .setInputCols(["document","token"])\
        .setOutputCol("word_embeddings")

sentence_embeddings = nlp.SentenceEmbeddings() \
        .setInputCols(["document", "word_embeddings"]) \
        .setOutputCol("sentence_embeddings") \
        .setPoolingStrategy("AVERAGE")

embeddings_pipeline = nlp.Pipeline(
    stages = [
        document_assembler,
        tokenizer,
        word_embeddings,
        sentence_embeddings,
    ])

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]


In [ ]:
trainingData_with_embeddings = embeddings_pipeline.fit(trainingData).transform(trainingData)\
                                                  .select("text","category","sentence_embeddings")
trainingData_with_embeddings.show(2,truncate=60)

+------------------------------------------------------------+--------+------------------------------------------------------------+
|                                                        text|category|                                         sentence_embeddings|
+------------------------------------------------------------+--------+------------------------------------------------------------+
| (4) The acute rehabilitation of a 21-year-old woman with...|     neg|[{sentence_embeddings, 0, 182,  (4) The acute rehabilitat...|
| A 16-year-old girl with erosive, polyarticular JRA showe...|     neg|[{sentence_embeddings, 0, 129,  A 16-year-old girl with e...|
+------------------------------------------------------------+--------+------------------------------------------------------------+
only showing top 2 rows



In [ ]:
testData_with_embeddings = embeddings_pipeline.fit(testData).transform(testData)\
                                                  .select("text","category","sentence_embeddings")
testData_with_embeddings.show(2,truncate=60)

+------------------------------------------------------------+--------+------------------------------------------------------------+
|                                                        text|category|                                         sentence_embeddings|
+------------------------------------------------------------+--------+------------------------------------------------------------+
| A 51-year-old-woman presented with chronic eosinophilia,...|     neg|[{sentence_embeddings, 0, 139,  A 51-year-old-woman prese...|
| Additionally, their Global Assessment Scale (GAS) score ...|     neg|[{sentence_embeddings, 0, 164,  Additionally, their Globa...|
+------------------------------------------------------------+--------+------------------------------------------------------------+
only showing top 2 rows



In [ ]:
log_folder="ADE_logs_healthcare_200d"

### ClassifierDL

In [ ]:
classifier_dl = nlp.ClassifierDLApproach()\
        .setInputCols(["sentence_embeddings"])\
        .setOutputCol("class")\
        .setLabelColumn("category")\
        .setBatchSize(16)\
        .setMaxEpochs(30)\
        .setLr(0.001)\
        .setDropout(0.3)\
        .setEnableOutputLogs(True)\
        .setOutputLogsPath(log_folder)

classifier_dl_pipeline = nlp.Pipeline(
    stages = [
        classifier_dl
    ])

In [ ]:
clfDL_model_hc200 = classifier_dl_pipeline.fit(trainingData_with_embeddings)

In [ ]:
preds = clfDL_model_hc200.transform(testData_with_embeddings)

preds_df = preds.select("category","text","class.result").toPandas()
preds_df["result"] = preds_df["result"].apply(lambda x : x[0])

print (classification_report(preds_df["category"], preds_df["result"]))

              precision    recall  f1-score   support

         neg       0.87      0.91      0.89      1031
         pos       0.76      0.68      0.72       423

    accuracy                           0.85      1454
   macro avg       0.82      0.80      0.81      1454
weighted avg       0.84      0.85      0.84      1454



In [ ]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["ClassifierDL_200d"] = [res["pos"]["f1-score"], res["accuracy"]]

### MultiClassifierDL

In [ ]:
# MultiClassifierDL accepts list of strings as input label. So we convert label column to array type.
trainingData_with_embeddings = trainingData_with_embeddings.withColumn("category_array", F.array(F.col("category")))
testData_with_embeddings = testData_with_embeddings.withColumn("category_array", F.array(F.col("category")))

In [ ]:
multiClassifier =nlp.MultiClassifierDLApproach()\
  .setInputCols("sentence_embeddings")\
  .setOutputCol("prediction")\
  .setLabelColumn("category_array")\
  .setBatchSize(32)\
  .setMaxEpochs(20)\
  .setLr(9e-3)\
  .setThreshold(0.5)\
  .setShufflePerEpoch(False)\
  .setEnableOutputLogs(True)\
  .setOutputLogsPath(log_folder)\
#   .setValidationSplit(0.1)

multiClassifier_pipeline = nlp.Pipeline(
    stages = [
        multiClassifier
    ])

In [ ]:
multiClassifier_model_hc200 = multiClassifier_pipeline.fit(trainingData_with_embeddings)

In [ ]:
preds = multiClassifier_model_hc200.transform(testData_with_embeddings)

In [ ]:
preds_df = preds.select("category","text","prediction.result","prediction.metadata").toPandas()
preds_df.result.apply(lambda x: str(x) ).value_counts()

['neg']    1017
['pos']     436
[]            1
Name: result, dtype: int64

In [ ]:
# We will get the highest score label as result. You can control the number of zero label results with setThreshold() in the training.
preds_df["scores"] = preds_df.metadata.apply(lambda x: {k:float(v) for k,v in x[0].items()} if len(x)>=1 else "")
preds_df["result"] = preds_df.scores.apply(lambda x: max(x, key=x.get) if len(x)>=1 else "")
preds_df = preds_df[(preds_df["result"]!="")]
preds_df["result"].value_counts()

neg    1017
pos     436
Name: result, dtype: int64

In [ ]:
print (classification_report(preds_df["category"], preds_df["result"]))

              precision    recall  f1-score   support

         neg       0.89      0.88      0.88      1031
         pos       0.71      0.73      0.72       422

    accuracy                           0.84      1453
   macro avg       0.80      0.81      0.80      1453
weighted avg       0.84      0.84      0.84      1453



In [ ]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["MultiClassifierDL_200d"] = [res["pos"]["f1-score"], res["accuracy"]]

### Generic Classifier

In [ ]:
# !pip install -q tensorflow==2.7.0 tensorflow_addons

In [ ]:
#from sparknlp_jsl.annotator import TFGraphBuilder

graph_folder = "gc_graph"

gc_graph_builder = medical.TFGraphBuilder()\
    .setModelName("generic_classifier")\
    .setInputCols(["features"])\
    .setLabelColumn("category")\
    .setHiddenLayers([300,200, 50])\
    .setHiddenAct("tanh")\
    .setHiddenActL2(True)\
    .setBatchNorm(True)\
    .setGraphFolder(graph_folder)\
    .setGraphFile("gcf_graph_200d.pb")

In [ ]:
features_asm =medical.FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("features")

gen_clf =medical.GenericClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("features")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/gcf_graph_200d.pb")\
    .setEpochsNumber(40)\
    .setBatchSize(16)\
    .setLearningRate(0.001)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.5)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = nlp.Pipeline(stages=[
    features_asm,
    gc_graph_builder,
    gen_clf])


In [ ]:
generic_model_hc200 = clf_Pipeline.fit(trainingData_with_embeddings)

TF Graph Builder configuration:
Model name: generic_classifier
Graph folder: gc_graph
Graph file name: gcf_graph_200d.pb
Build params: {'input_dim': 200, 'output_dim': 2, 'hidden_layers': [300, 200, 50], 'hidden_act': 'tanh', 'hidden_act_l2': True, 'batch_norm': True}
generic_classifier graph exported to gc_graph/gcf_graph_200d.pb


In [ ]:
pred_df = generic_model_hc200.transform(testData_with_embeddings)

preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

              precision    recall  f1-score   support

         neg       0.93      0.73      0.82      1031
         pos       0.57      0.87      0.69       423

    accuracy                           0.77      1454
   macro avg       0.75      0.80      0.76      1454
weighted avg       0.83      0.77      0.78      1454



In [ ]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["GenericClassifier_200d"] = [res["pos"]["f1-score"], res["accuracy"]]

### GenericLogRegClassifier

In [ ]:
graph_folder = "gc_graph"

gc_logreg_graph_builder = medical.TFGraphBuilder()\
    .setModelName("logreg_classifier")\
    .setInputCols(["feature_vector"]) \
    .setLabelColumn("category")\
    .setGraphFolder(graph_folder)\
    .setGraphFile("log_reg_graph.pb")

GenericLogRegClassifier needs outputs from FeaturesAssembler. The FeaturesAssembler is used to collect features from different columns or an embeddings column.

The GenericLogRegClassifier takes FEATURE_VECTOR annotations as input, classifies them and outputs CATEGORY annotations.

In [ ]:
features_asm = medical.FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("feature_vector")

gen_clf = medical.GenericLogRegClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("feature_vector")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/log_reg_graph.pb")\
    .setEpochsNumber(20)\
    .setBatchSize(128)\
    .setLearningRate(0.01)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.1)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = nlp.Pipeline(stages=[
    features_asm,
    gc_logreg_graph_builder,
    gen_clf])


In [ ]:
generic_model_hc200 = clf_Pipeline.fit(trainingData_with_embeddings)

TF Graph Builder configuration:
Model name: logreg_classifier
Graph folder: gc_graph
Graph file name: log_reg_graph.pb
Build params: {'input_dim': 200, 'output_dim': 2, 'hidden_layers': [], 'output_act': 'sigmoid'}
logreg_classifier graph exported to gc_graph/log_reg_graph.pb


In [ ]:
pred_df = generic_model_hc200.transform(testData_with_embeddings)

In [ ]:
preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

              precision    recall  f1-score   support

         neg       0.80      0.94      0.86      1031
         pos       0.74      0.43      0.54       423

    accuracy                           0.79      1454
   macro avg       0.77      0.68      0.70      1454
weighted avg       0.78      0.79      0.77      1454



In [ ]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["GenericLogReg_200d"] = [res["pos"]["f1-score"], res["accuracy"]]

### GenericSVMClassifier

In [ ]:
graph_folder = "gc_graph"

gc_svm_graph_builder = medical.TFGraphBuilder()\
    .setModelName("svm_classifier")\
    .setInputCols(["feature_vector"]) \
    .setLabelColumn("category")\
    .setGraphFolder(graph_folder)\
    .setGraphFile("svm_graph.pb")

GenericSVMClassifier needs outputs from FeaturesAssembler. The FeaturesAssembler is used to collect features from different columns or an embeddings column.

The GenericSVMClassifier takes FEATURE_VECTOR annotations as input, classifies them and outputs CATEGORY annotations.

In [ ]:
features_asm = medical.FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("feature_vector")

gen_clf = medical.GenericSVMClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("feature_vector")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/svm_graph.pb")\
    .setEpochsNumber(20)\
    .setBatchSize(128)\
    .setLearningRate(0.005)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.1)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = nlp.Pipeline(stages=[
    features_asm,
    gc_svm_graph_builder,
    gen_clf])


In [ ]:
generic_model_hc200 = clf_Pipeline.fit(trainingData_with_embeddings)

TF Graph Builder configuration:
Model name: svm_classifier
Graph folder: gc_graph
Graph file name: svm_graph.pb
Build params: {'input_dim': 200, 'output_dim': 2, 'hidden_layers': [], 'output_act': 'sigmoid', 'loss_func': 'hinge'}
svm_classifier graph exported to gc_graph/svm_graph.pb


In [ ]:
pred_df = generic_model_hc200.transform(testData_with_embeddings)

In [ ]:
preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

              precision    recall  f1-score   support

         neg       0.82      0.92      0.87      1031
         pos       0.73      0.50      0.59       423

    accuracy                           0.80      1454
   macro avg       0.77      0.71      0.73      1454
weighted avg       0.79      0.80      0.79      1454



In [ ]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["GenericSVM_200d"] = [res["pos"]["f1-score"], res["accuracy"]]

## Bert Sentence Embeddings (sbiobert_base_cased_mli)

Now we will extract [sbiobert_base_cased_mli](https://nlp.johnsnowlabs.com/2020/11/27/sbiobert_base_cased_mli_en.html) embeddings which has 768 dimension output and use this embeddings in the model training.

In [ ]:
document_assembler = nlp.DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

bert_sent = nlp.BertSentenceEmbeddings.pretrained("sbiobert_base_cased_mli", 'en','clinical/models')\
    .setInputCols(["document"])\
    .setOutputCol("sentence_embeddings")

embeddings_pipeline = nlp.Pipeline(
    stages = [document_assembler,
              bert_sent])

sbiobert_base_cased_mli download started this may take some time.
Approximate size to download 384.3 MB
[OK!]


In [ ]:
trainingData_with_embeddings = embeddings_pipeline.fit(trainingData).transform(trainingData)\
                                                  .select("text","category","sentence_embeddings")

trainingData_with_embeddings.show(2,truncate=60)

+------------------------------------------------------------+--------+------------------------------------------------------------+
|                                                        text|category|                                         sentence_embeddings|
+------------------------------------------------------------+--------+------------------------------------------------------------+
| (4) The acute rehabilitation of a 21-year-old woman with...|     neg|[{sentence_embeddings, 0, 182,  (4) The acute rehabilitat...|
| A 16-year-old girl with erosive, polyarticular JRA showe...|     neg|[{sentence_embeddings, 0, 129,  A 16-year-old girl with e...|
+------------------------------------------------------------+--------+------------------------------------------------------------+
only showing top 2 rows



In [ ]:
testData_with_embeddings = embeddings_pipeline.fit(testData).transform(testData)\
                                                  .select("text","category","sentence_embeddings")

testData_with_embeddings.show(2,truncate=60)

+------------------------------------------------------------+--------+------------------------------------------------------------+
|                                                        text|category|                                         sentence_embeddings|
+------------------------------------------------------------+--------+------------------------------------------------------------+
| A 51-year-old-woman presented with chronic eosinophilia,...|     neg|[{sentence_embeddings, 0, 139,  A 51-year-old-woman prese...|
| Additionally, their Global Assessment Scale (GAS) score ...|     neg|[{sentence_embeddings, 0, 164,  Additionally, their Globa...|
+------------------------------------------------------------+--------+------------------------------------------------------------+
only showing top 2 rows



In [ ]:
log_folder="ADE_logs_bert"

### ClassifierDL

In [ ]:
classifier_dl = nlp.ClassifierDLApproach()\
        .setInputCols(["sentence_embeddings"])\
        .setOutputCol("class")\
        .setLabelColumn("category")\
        .setBatchSize(32)\
        .setMaxEpochs(20)\
        .setLr(0.001)\
        .setDropout(0.3)\
        .setEnableOutputLogs(True)\
        .setOutputLogsPath(log_folder)

classifier_dl_pipeline = nlp.Pipeline(stages=[classifier_dl])

In [ ]:
clfDL_model_bert = classifier_dl_pipeline.fit(trainingData_with_embeddings)

In [ ]:
preds = clfDL_model_bert.transform(testData_with_embeddings)

preds_df = preds.select("category","text","class.result").toPandas()
preds_df["result"] = preds_df["result"].apply(lambda x : x[0])

print (classification_report(preds_df["category"], preds_df["result"]))

              precision    recall  f1-score   support

         neg       0.71      1.00      0.83      1031
         pos       0.00      0.00      0.00       423

    accuracy                           0.71      1454
   macro avg       0.35      0.50      0.41      1454
weighted avg       0.50      0.71      0.59      1454



In [ ]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["ClassifierDL_bert"] = [res["pos"]["f1-score"], res["accuracy"]]

### MultiClassifierDL

In [ ]:
# MultiClassifierDL accepts list of strings as input label. So we convert label column to array type.
trainingData_with_embeddings = trainingData_with_embeddings.withColumn("category_array", F.array(F.col("category")))
testData_with_embeddings = testData_with_embeddings.withColumn("category_array", F.array(F.col("category")))

In [ ]:
multiClassifier =nlp.MultiClassifierDLApproach()\
  .setInputCols("sentence_embeddings")\
  .setOutputCol("prediction")\
  .setLabelColumn("category_array")\
  .setBatchSize(32)\
  .setMaxEpochs(20)\
  .setLr(9e-3)\
  .setThreshold(0.5)\
  .setShufflePerEpoch(False)\
  .setEnableOutputLogs(True)\
  .setOutputLogsPath(log_folder)\
#   .setValidationSplit(0.1)

multiClassifier_pipeline = nlp.Pipeline(
    stages = [
        multiClassifier
    ])

In [ ]:
multiClassifier_model_bert = multiClassifier_pipeline.fit(trainingData_with_embeddings)

In [ ]:
preds = multiClassifier_model_bert.transform(testData_with_embeddings)

In [ ]:
preds_df = preds.select("category","text","prediction.result","prediction.metadata").toPandas()
preds_df.result.apply(lambda x: str(x) ).value_counts()

['neg']    1067
['pos']     385
[]            2
Name: result, dtype: int64

In [ ]:
# We will get the highest score label as result. You can control the number of zero label results with setThreshold() in the training.
preds_df["scores"] = preds_df.metadata.apply(lambda x: {k:float(v) for k,v in x[0].items()} if len(x)>=1 else "")
preds_df["result"] = preds_df.scores.apply(lambda x: max(x, key=x.get) if len(x)>=1 else "")
preds_df = preds_df[(preds_df["result"]!="")]
preds_df["result"].value_counts()

neg    1067
pos     385
Name: result, dtype: int64

In [ ]:
print (classification_report(preds_df["category"], preds_df["result"]))

              precision    recall  f1-score   support

         neg       0.89      0.93      0.91      1029
         pos       0.80      0.73      0.76       423

    accuracy                           0.87      1452
   macro avg       0.85      0.83      0.84      1452
weighted avg       0.87      0.87      0.87      1452



In [ ]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["MultiClassifierDL_bert"] = [res["pos"]["f1-score"], res["accuracy"]]

### Generic Classifier

In [ ]:
# !pip install -q tensorflow==2.7.0 tensorflow_addons

In [ ]:
from sparknlp_jsl.annotator import TFGraphBuilder

graph_folder = "gc_graph"

gc_graph_builder =medical.TFGraphBuilder()\
    .setModelName("generic_classifier")\
    .setInputCols(["features"])\
    .setLabelColumn("category")\
    .setHiddenLayers([300,200, 50])\
    .setHiddenAct("tanh")\
    .setHiddenActL2(True)\
    .setBatchNorm(True)\
    .setGraphFolder(graph_folder)\
    .setGraphFile("gcf_graph_bert.pb")

In [ ]:
features_asm =medical.FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("features")

gen_clf =medical.GenericClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("features")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/gcf_graph_bert.pb")\
    .setEpochsNumber(20)\
    .setBatchSize(32)\
    .setLearningRate(0.001)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.5)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = nlp.Pipeline(stages=[
    features_asm,
    gc_graph_builder,
    gen_clf])


In [ ]:
generic_model_bert = clf_Pipeline.fit(trainingData_with_embeddings)

TF Graph Builder configuration:
Model name: generic_classifier
Graph folder: gc_graph
Graph file name: gcf_graph_bert.pb
Build params: {'input_dim': 768, 'output_dim': 2, 'hidden_layers': [300, 200, 50], 'hidden_act': 'tanh', 'hidden_act_l2': True, 'batch_norm': True}
generic_classifier graph exported to gc_graph/gcf_graph_bert.pb


In [ ]:
pred_df = generic_model_bert.transform(testData_with_embeddings)

preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

              precision    recall  f1-score   support

         neg       0.94      0.89      0.91      1031
         pos       0.76      0.86      0.81       423

    accuracy                           0.88      1454
   macro avg       0.85      0.87      0.86      1454
weighted avg       0.89      0.88      0.88      1454



In [ ]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["GenericClassifier_bert"] = [res["pos"]["f1-score"], res["accuracy"]]

### GenericLogRegClassifier

In [ ]:
graph_folder = "gc_graph"

gc_logreg_graph_builder = medical.TFGraphBuilder()\
    .setModelName("logreg_classifier")\
    .setInputCols(["feature_vector"]) \
    .setLabelColumn("category")\
    .setGraphFolder(graph_folder)\
    .setGraphFile("log_reg_graph.pb")

GenericLogRegClassifier needs outputs from FeaturesAssembler. The FeaturesAssembler is used to collect features from different columns or an embeddings column.

The GenericLogRegClassifier takes FEATURE_VECTOR annotations as input, classifies them and outputs CATEGORY annotations.

In [ ]:
features_asm = medical.FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("feature_vector")

gen_clf = medical.GenericLogRegClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("feature_vector")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/log_reg_graph.pb")\
    .setEpochsNumber(20)\
    .setBatchSize(128)\
    .setLearningRate(0.01)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.1)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = nlp.Pipeline(stages=[
    features_asm,
    gc_logreg_graph_builder,
    gen_clf])


In [ ]:
generic_model_bert = clf_Pipeline.fit(trainingData_with_embeddings)

TF Graph Builder configuration:
Model name: logreg_classifier
Graph folder: gc_graph
Graph file name: log_reg_graph.pb
Build params: {'input_dim': 768, 'output_dim': 2, 'hidden_layers': [], 'output_act': 'sigmoid'}
logreg_classifier graph exported to gc_graph/log_reg_graph.pb


In [ ]:
pred_df = generic_model_bert.transform(testData_with_embeddings)

In [ ]:
preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

              precision    recall  f1-score   support

         neg       0.88      0.94      0.91      1031
         pos       0.83      0.68      0.75       423

    accuracy                           0.87      1454
   macro avg       0.85      0.81      0.83      1454
weighted avg       0.86      0.87      0.86      1454



In [ ]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["GenericLogReg_bert"] = [res["pos"]["f1-score"], res["accuracy"]]

### GenericSVMClassifier

In [ ]:
graph_folder = "gc_graph"

gc_svm_graph_builder = medical.TFGraphBuilder()\
    .setModelName("svm_classifier")\
    .setInputCols(["feature_vector"]) \
    .setLabelColumn("category")\
    .setGraphFolder(graph_folder)\
    .setGraphFile("svm_graph.pb")

GenericSVMClassifier needs outputs from FeaturesAssembler. The FeaturesAssembler is used to collect features from different columns or an embeddings column.

The GenericSVMClassifier takes FEATURE_VECTOR annotations as input, classifies them and outputs CATEGORY annotations.

In [ ]:
features_asm = medical.FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("feature_vector")

gen_clf = medical.GenericSVMClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("feature_vector")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/svm_graph.pb")\
    .setEpochsNumber(20)\
    .setBatchSize(128)\
    .setLearningRate(0.005)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.1)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = nlp.Pipeline(stages=[
    features_asm,
    gc_svm_graph_builder,
    gen_clf])


In [ ]:
generic_model_bert = clf_Pipeline.fit(trainingData_with_embeddings)

TF Graph Builder configuration:
Model name: svm_classifier
Graph folder: gc_graph
Graph file name: svm_graph.pb
Build params: {'input_dim': 768, 'output_dim': 2, 'hidden_layers': [], 'output_act': 'sigmoid', 'loss_func': 'hinge'}
svm_classifier graph exported to gc_graph/svm_graph.pb


In [ ]:
pred_df = generic_model_bert.transform(testData_with_embeddings)

In [ ]:
preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

              precision    recall  f1-score   support

         neg       0.86      0.96      0.91      1031
         pos       0.87      0.61      0.72       423

    accuracy                           0.86      1454
   macro avg       0.86      0.79      0.81      1454
weighted avg       0.86      0.86      0.85      1454



In [ ]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["GenericSVM_bert"] = [res["pos"]["f1-score"], res["accuracy"]]

## DocumentLogRegClassifier

In [ ]:
document_assembler = nlp.DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

tokenizer = nlp.Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

normalizer = nlp.Normalizer() \
      .setInputCols(["token"]) \
      .setOutputCol("normalized")

stopwords_cleaner = nlp.StopWordsCleaner()\
      .setInputCols("normalized")\
      .setOutputCol("cleanTokens")\
      .setCaseSensitive(False)

stemmer = nlp.Stemmer() \
      .setInputCols(["cleanTokens"]) \
      .setOutputCol("stem")

logreg = medical.DocumentLogRegClassifierApproach()\
    .setInputCols("stem")\
    .setLabelCol("category")\
    .setOutputCol("prediction")\
    .setMaxIter(3)\
    .setTol(1e-6)

clf_Pipeline = nlp.Pipeline(stages=[
    document_assembler,
    tokenizer,
    normalizer,
    stopwords_cleaner,
    stemmer,
    logreg])
doclogreg_model = clf_Pipeline.fit(trainingData)

In [ ]:
pred_df = doclogreg_model.transform(testData)

preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

              precision    recall  f1-score   support

         neg       0.85      0.93      0.89      1031
         pos       0.78      0.61      0.68       423

    accuracy                           0.84      1454
   macro avg       0.82      0.77      0.79      1454
weighted avg       0.83      0.84      0.83      1454



In [ ]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["DocumentLogRegClassifier"] = [res["pos"]["f1-score"], res["accuracy"]]

## Comparision of All Models' Result in ADE Dataset

Here is the all results of classifier models. Only positive ADE f1 scores and accuracy scores are presented.

In [ ]:
results_df

,pos-f1-score,accuracy
ClassifierDL_100d,0.661458,0.821183
MultiClassifierDL_100d,0.654971,0.796972
GenericClassifier_100d,0.651357,0.770289
GenericLogReg_100d,0.451713,0.757909
GenericSVM_100d,0.434783,0.758597
ClassifierDL_200d,0.720000,0.845942
MultiClassifierDL_200d,0.722611,0.836201
GenericClassifier_200d,0.691011,0.773040
GenericLogReg_200d,0.541106,0.788858
GenericSVM_200d,0.593007,0.799862


# Mtsamples Dataset

**❗Please restart runtime and continue**

In [ ]:
from johnsnowlabs import nlp, medical, visual
import pandas as pd
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import pyspark.sql.functions as F

# Automatically load license data and start a session with all jars user has access to
spark = nlp.start(hardware_target="gpu")

Spark Session already created, some configs may not take.
👌 Detected license file /content/5.0.0.spark_nlp_for_healthcare.json


## Load Dataset

In [ ]:
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/data/mtsamples_classifier.csv

In [ ]:
spark_df = spark.read.csv("mtsamples_classifier.csv", header = True)

spark_df.show(10,truncate=100)

+----------------+----------------------------------------------------------------------------------------------------+
|        category|                                                                                                text|
+----------------+----------------------------------------------------------------------------------------------------+
|Gastroenterology| PROCEDURES PERFORMED: Colonoscopy. INDICATIONS: Renewed symptoms likely consistent with active f...|
|Gastroenterology| OPERATION 1. Ivor-Lewis esophagogastrectomy. 2. Feeding jejunostomy. 3. Placement of two right-s...|
|Gastroenterology| PREOPERATIVE DIAGNOSES: 1. Gastroesophageal reflux disease. 2. Chronic dyspepsia. POSTOPERATIVE ...|
|Gastroenterology| PROCEDURE: Colonoscopy. PREOPERATIVE DIAGNOSES: Rectal bleeding and perirectal abscess. POSTOPER...|
|Gastroenterology| PREOPERATIVE DIAGNOSIS: Right colon tumor. POSTOPERATIVE DIAGNOSES: 1. Right colon cancer. 2. As...|
|Gastroenterology| PREOPERATIVE DIAGNOSI

In [ ]:
spark_df.printSchema()

root
 |-- category: string (nullable = true)
 |-- text: string (nullable = true)



In [ ]:
spark_df.count()

638

In [ ]:
spark_df.groupBy("category").count().show()

+----------------+-----+
|        category|count|
+----------------+-----+
|         Urology|  115|
|       Neurology|  143|
|      Orthopedic|  223|
|Gastroenterology|  157|
+----------------+-----+



In [ ]:
(trainingData, testData) = spark_df.randomSplit([0.8, 0.2], seed = 42)

In [ ]:
trainingData.groupBy("category").count().show()

+----------------+-----+
|        category|count|
+----------------+-----+
|         Urology|   95|
|       Neurology|  121|
|      Orthopedic|  188|
|Gastroenterology|  132|
+----------------+-----+



In [ ]:
testData.groupBy("category").count().show()

+----------------+-----+
|        category|count|
+----------------+-----+
|         Urology|   20|
|       Neurology|   22|
|      Orthopedic|   35|
|Gastroenterology|   25|
+----------------+-----+



## 100 Dimension Healthcare Embeddings (embeddings_healthcare_100d)



We will extract [healthcare_100d embeddings](https://nlp.johnsnowlabs.com/2020/05/29/embeddings_healthcare_100d_en.html) and use it in the classificaiton model training.

In [ ]:
document_assembler = nlp.DocumentAssembler()\
        .setInputCol("text")\
        .setOutputCol("document")

tokenizer = nlp.Tokenizer() \
            .setInputCols(["document"]) \
            .setOutputCol("token")

word_embeddings = nlp.WordEmbeddingsModel.pretrained("embeddings_healthcare_100d","en","clinical/models")\
        .setInputCols(["document","token"])\
        .setOutputCol("word_embeddings")

sentence_embeddings = nlp.SentenceEmbeddings() \
        .setInputCols(["document", "word_embeddings"]) \
        .setOutputCol("sentence_embeddings") \
        .setPoolingStrategy("AVERAGE")

embeddings_pipeline = nlp.Pipeline(
    stages = [
        document_assembler,
        tokenizer,
        word_embeddings,
        sentence_embeddings,

    ])

embeddings_healthcare_100d download started this may take some time.
Approximate size to download 475.8 MB
[OK!]


In [ ]:
trainingData_with_embeddings = embeddings_pipeline.fit(trainingData).transform(trainingData)
trainingData_with_embeddings = trainingData_with_embeddings.select("text","category","sentence_embeddings")
trainingData_with_embeddings.show(2,truncate=60)

+------------------------------------------------------------+----------------+------------------------------------------------------------+
|                                                        text|        category|                                         sentence_embeddings|
+------------------------------------------------------------+----------------+------------------------------------------------------------+
| ADMISSION DIAGNOSIS: Symptomatic cholelithiasis. DISCHAR...|Gastroenterology|[{sentence_embeddings, 0, 2228,  ADMISSION DIAGNOSIS: Sym...|
| ADMITTING DIAGNOSES: Hiatal hernia, gastroesophageal ref...|Gastroenterology|[{sentence_embeddings, 0, 3237,  ADMITTING DIAGNOSES: Hia...|
+------------------------------------------------------------+----------------+------------------------------------------------------------+
only showing top 2 rows



In [ ]:
testData_with_embeddings = embeddings_pipeline.fit(testData).transform(testData)
testData_with_embeddings = testData_with_embeddings.select("text","category","sentence_embeddings")
testData_with_embeddings.show(2,truncate=60)

+------------------------------------------------------------+----------------+------------------------------------------------------------+
|                                                        text|        category|                                         sentence_embeddings|
+------------------------------------------------------------+----------------+------------------------------------------------------------+
| ADMITTING DIAGNOSIS: Gastrointestinal bleed. HISTORY OF ...|Gastroenterology|[{sentence_embeddings, 0, 3978,  ADMITTING DIAGNOSIS: Gas...|
| CHIEF COMPLAINT: Dysphagia and hematemesis while vomitin...|Gastroenterology|[{sentence_embeddings, 0, 6515,  CHIEF COMPLAINT: Dysphag...|
+------------------------------------------------------------+----------------+------------------------------------------------------------+
only showing top 2 rows



In [ ]:
testData_with_embeddings.printSchema()

root
 |-- text: string (nullable = true)
 |-- category: string (nullable = true)
 |-- sentence_embeddings: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)



In [ ]:
testData_with_embeddings.select(testData_with_embeddings.sentence_embeddings.embeddings).show(3,truncate=120)

+------------------------------------------------------------------------------------------------------------------------+
|                                                                                          sentence_embeddings.embeddings|
+------------------------------------------------------------------------------------------------------------------------+
|[[-0.010433162, 0.0127568655, 0.110687375, 0.1609855, -0.14818177, -0.050856464, 0.064748034, -0.019294674, -0.002178...|
|[[-0.0055608996, 0.018396137, 0.12305377, 0.13861515, -0.18651922, -0.07559964, 0.12978752, -0.0046452526, -0.0031408...|
|[[-0.019461514, 0.060639214, 0.11142021, 0.12826534, -0.16216077, -0.10416892, 0.17136306, -0.020363769, 0.01169337, ...|
+------------------------------------------------------------------------------------------------------------------------+
only showing top 3 rows



In [ ]:
log_folder="Mt_logs_healthcare_100d"
!mkdir -p $log_folder

### ClassifierDL

In [ ]:
classifier_dl = nlp.ClassifierDLApproach()\
        .setInputCols(["sentence_embeddings"])\
        .setOutputCol("prediction")\
        .setLabelColumn("category")\
        .setBatchSize(8)\
        .setMaxEpochs(50)\
        .setLr(0.005)\
        .setDropout(0.3)\
        .setEnableOutputLogs(True)\
        .setOutputLogsPath(log_folder)\
        # .setValidationSplit(0.1)

classifier_dl_pipeline = nlp.Pipeline(stages=[classifier_dl])

In [ ]:
clfDL_model_hc100 = classifier_dl_pipeline.fit(trainingData_with_embeddings)

In [ ]:
preds = clfDL_model_hc100.transform(testData_with_embeddings)

In [ ]:
preds_df = preds.select("category","text","prediction.result").toPandas()
preds_df["result"] = preds_df["result"].apply(lambda x : x[0])

print (classification_report(preds_df["category"], preds_df["result"]))

                  precision    recall  f1-score   support

Gastroenterology       0.73      0.76      0.75        25
       Neurology       0.71      0.68      0.70        22
      Orthopedic       0.87      0.94      0.90        35
         Urology       0.53      0.45      0.49        20

        accuracy                           0.75       102
       macro avg       0.71      0.71      0.71       102
    weighted avg       0.73      0.75      0.74       102



In [ ]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df = pd.DataFrame(columns=["macro-f1-score","weighted-f1-score","accuracy"])
results_df.loc["ClassifierDL_100d"] = [res["macro avg"]["f1-score"], res["weighted avg"]["f1-score"], res["accuracy"]]
results_df

,macro-f1-score,weighted-f1-score,accuracy
ClassifierDL_100d,0.708342,0.738724,0.745098


### MultiClassifierDL

In [ ]:
# MultiClassifierDL accepts list of strings as input label. So we convert label column to array type.
trainingData_with_embeddings = trainingData_with_embeddings.withColumn("category_array", F.array(F.col("category")))
testData_with_embeddings = testData_with_embeddings.withColumn("category_array", F.array(F.col("category")))

In [ ]:
multiClassifier =nlp.MultiClassifierDLApproach()\
  .setInputCols("sentence_embeddings")\
  .setOutputCol("prediction")\
  .setLabelColumn("category_array")\
  .setBatchSize(8)\
  .setMaxEpochs(40)\
  .setLr(5e-3)\
  .setThreshold(0.5)\
  .setShufflePerEpoch(False)\
  .setEnableOutputLogs(True)\
  .setOutputLogsPath(log_folder)\
#   .setValidationSplit(0.1)

multiClassifier_pipeline = nlp.Pipeline(
    stages = [
        multiClassifier
    ])

In [ ]:
multiClassifier_model_hc100 = multiClassifier_pipeline.fit(trainingData_with_embeddings)

In [ ]:
preds = multiClassifier_model_hc100.transform(testData_with_embeddings)

In [ ]:
preds_df = preds.select("category","text","prediction.result","prediction.metadata").toPandas()
preds_df.result.apply(lambda x: str(x) ).value_counts()

['Orthopedic']                        35
['Urology']                           21
['Gastroenterology']                  19
['Neurology']                         17
[]                                     5
['Neurology', 'Orthopedic']            4
['Orthopedic', 'Gastroenterology']     1
Name: result, dtype: int64

In [ ]:
# We will get the highest score label as result. You can control the number of zero label results with setThreshold() in the training.
preds_df["scores"] = preds_df.metadata.apply(lambda x: {k:float(v) for k,v in x[0].items()} if len(x)>=1 else "")
preds_df["result"] = preds_df.scores.apply(lambda x: max(x, key=x.get) if len(x)>=1 else "")
preds_df = preds_df[(preds_df["result"]!="")]
preds_df["result"].value_counts()

Orthopedic          36
Urology             21
Gastroenterology    20
Neurology           20
Name: result, dtype: int64

In [ ]:
print (classification_report(preds_df["category"], preds_df["result"]))

                  precision    recall  f1-score   support

Gastroenterology       0.95      0.86      0.90        22
       Neurology       0.80      0.76      0.78        21
      Orthopedic       0.89      0.91      0.90        35
         Urology       0.90      1.00      0.95        19

        accuracy                           0.89        97
       macro avg       0.89      0.88      0.88        97
    weighted avg       0.89      0.89      0.89        97



In [ ]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["MultiClassifierDL_100d"] = [res["macro avg"]["f1-score"], res["weighted avg"]["f1-score"], res["accuracy"]]

### Generic Classifier

In [ ]:
# !pip install -q tensorflow==2.7.0 tensorflow_addons

In [ ]:
from sparknlp_jsl.annotator import TFGraphBuilder

graph_folder = "gc_graph"

gc_graph_builder =medical.TFGraphBuilder()\
    .setModelName("generic_classifier")\
    .setInputCols(["features"])\
    .setLabelColumn("category")\
    .setHiddenLayers([300,200, 50])\
    .setHiddenAct("tanh")\
    .setHiddenActL2(True)\
    .setBatchNorm(True)\
    .setGraphFolder(graph_folder)\
    .setGraphFile("gcf_graph_100d.pb")

In [ ]:
features_asm =medical.FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("features")

gen_clf =medical.GenericClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("features")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/gcf_graph_100d.pb")\
    .setEpochsNumber(50)\
    .setBatchSize(32)\
    .setLearningRate(0.001)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.3)\
    .setFixImbalance(False)\
    # .setValidationSplit(0.1)

clf_Pipeline = nlp.Pipeline(stages=[
    features_asm,
    gc_graph_builder,
    gen_clf])


In [ ]:
generic_model_hc100 = clf_Pipeline.fit(trainingData_with_embeddings)

TF Graph Builder configuration:
Model name: generic_classifier
Graph folder: gc_graph
Graph file name: gcf_graph_100d.pb
Build params: {'input_dim': 100, 'output_dim': 4, 'hidden_layers': [300, 200, 50], 'hidden_act': 'tanh', 'hidden_act_l2': True, 'batch_norm': True}
generic_classifier graph exported to gc_graph/gcf_graph_100d.pb


In [ ]:
pred_df = generic_model_hc100.transform(testData_with_embeddings)

In [ ]:
preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

                  precision    recall  f1-score   support

Gastroenterology       0.96      0.88      0.92        25
       Neurology       0.78      0.82      0.80        22
      Orthopedic       0.89      0.91      0.90        35
         Urology       0.90      0.90      0.90        20

        accuracy                           0.88       102
       macro avg       0.88      0.88      0.88       102
    weighted avg       0.88      0.88      0.88       102



In [ ]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["GenericClassifier_100d"] = [res["macro avg"]["f1-score"], res["weighted avg"]["f1-score"], res["accuracy"]]

### GenericLogRegClassifier

In [ ]:
graph_folder = "gc_graph"

gc_logreg_graph_builder = medical.TFGraphBuilder()\
    .setModelName("logreg_classifier")\
    .setInputCols(["feature_vector"]) \
    .setLabelColumn("category")\
    .setGraphFolder(graph_folder)\
    .setGraphFile("log_reg_graph.pb")

GenericLogRegClassifier needs outputs from FeaturesAssembler. The FeaturesAssembler is used to collect features from different columns or an embeddings column.

The GenericLogRegClassifier takes FEATURE_VECTOR annotations as input, classifies them and outputs CATEGORY annotations.

In [ ]:
features_asm = medical.FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("feature_vector")

gen_clf = medical.GenericLogRegClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("feature_vector")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/log_reg_graph.pb")\
    .setEpochsNumber(45)\
    .setBatchSize(32)\
    .setLearningRate(0.02)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.1)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = nlp.Pipeline(stages=[
    features_asm,
    gc_logreg_graph_builder,
    gen_clf])


In [ ]:
generic_model_hc100 = clf_Pipeline.fit(trainingData_with_embeddings)

TF Graph Builder configuration:
Model name: logreg_classifier
Graph folder: gc_graph
Graph file name: log_reg_graph.pb
Build params: {'input_dim': 100, 'output_dim': 4, 'hidden_layers': [], 'output_act': 'sigmoid'}
logreg_classifier graph exported to gc_graph/log_reg_graph.pb


In [ ]:
pred_df = generic_model_hc100.transform(testData_with_embeddings)

In [ ]:
preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

                  precision    recall  f1-score   support

Gastroenterology       0.92      0.88      0.90        25
       Neurology       0.81      0.77      0.79        22
      Orthopedic       0.84      0.91      0.88        35
         Urology       0.89      0.85      0.87        20

        accuracy                           0.86       102
       macro avg       0.87      0.85      0.86       102
    weighted avg       0.86      0.86      0.86       102



In [ ]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["GenericLogReg_100d"] = [res["macro avg"]["f1-score"], res["weighted avg"]["f1-score"], res["accuracy"]]

### GenericSVMClassifier

In [ ]:
graph_folder = "gc_graph"

gc_svm_graph_builder = medical.TFGraphBuilder()\
    .setModelName("svm_classifier")\
    .setInputCols(["feature_vector"]) \
    .setLabelColumn("category")\
    .setGraphFolder(graph_folder)\
    .setGraphFile("svm_graph.pb")

GenericSVMClassifier needs outputs from FeaturesAssembler. The FeaturesAssembler is used to collect features from different columns or an embeddings column.

The GenericSVMClassifier takes FEATURE_VECTOR annotations as input, classifies them and outputs CATEGORY annotations.

In [ ]:
features_asm = medical.FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("feature_vector")

gen_clf = medical.GenericSVMClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("feature_vector")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/svm_graph.pb")\
    .setEpochsNumber(45)\
    .setBatchSize(32)\
    .setLearningRate(0.02)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.1)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = nlp.Pipeline(stages=[
    features_asm,
    gc_svm_graph_builder,
    gen_clf])


In [ ]:
generic_model_hc100 = clf_Pipeline.fit(trainingData_with_embeddings)

TF Graph Builder configuration:
Model name: svm_classifier
Graph folder: gc_graph
Graph file name: svm_graph.pb
Build params: {'input_dim': 100, 'output_dim': 4, 'hidden_layers': [], 'output_act': 'sigmoid', 'loss_func': 'hinge'}
svm_classifier graph exported to gc_graph/svm_graph.pb


In [ ]:
pred_df = generic_model_hc100.transform(testData_with_embeddings)

In [ ]:
preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

                  precision    recall  f1-score   support

Gastroenterology       0.92      0.88      0.90        25
       Neurology       0.82      0.82      0.82        22
      Orthopedic       0.91      0.91      0.91        35
         Urology       0.86      0.90      0.88        20

        accuracy                           0.88       102
       macro avg       0.88      0.88      0.88       102
    weighted avg       0.88      0.88      0.88       102



In [ ]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["GenericSVM_100d"] = [res["macro avg"]["f1-score"], res["weighted avg"]["f1-score"], res["accuracy"]]

## 200 Dimension Healthcare Embeddings (embeddings_clinical)



Now we will extract [embeddings_clinical](https://nlp.johnsnowlabs.com/2020/01/28/embeddings_clinical_en.html) embeddings which has 200 dimension output and use this embeddings in the model training.

In [ ]:
document_assembler = nlp.DocumentAssembler()\
        .setInputCol("text")\
        .setOutputCol("document")

tokenizer = nlp.Tokenizer() \
            .setInputCols(["document"]) \
            .setOutputCol("token")

word_embeddings = nlp.WordEmbeddingsModel.pretrained("embeddings_clinical","en","clinical/models")\
        .setInputCols(["document","token"])\
        .setOutputCol("word_embeddings")

sentence_embeddings = nlp.SentenceEmbeddings() \
        .setInputCols(["document", "word_embeddings"]) \
        .setOutputCol("sentence_embeddings") \
        .setPoolingStrategy("AVERAGE")

embeddings_pipeline = nlp.Pipeline(
    stages = [
        document_assembler,
        tokenizer,
        word_embeddings,
        sentence_embeddings,
    ])

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]


In [ ]:
trainingData_with_embeddings = embeddings_pipeline.fit(trainingData).transform(trainingData)\
                                                  .select("text","category","sentence_embeddings")
trainingData_with_embeddings.show(2,truncate=60)

+------------------------------------------------------------+----------------+------------------------------------------------------------+
|                                                        text|        category|                                         sentence_embeddings|
+------------------------------------------------------------+----------------+------------------------------------------------------------+
| ADMISSION DIAGNOSIS: Symptomatic cholelithiasis. DISCHAR...|Gastroenterology|[{sentence_embeddings, 0, 2228,  ADMISSION DIAGNOSIS: Sym...|
| ADMITTING DIAGNOSES: Hiatal hernia, gastroesophageal ref...|Gastroenterology|[{sentence_embeddings, 0, 3237,  ADMITTING DIAGNOSES: Hia...|
+------------------------------------------------------------+----------------+------------------------------------------------------------+
only showing top 2 rows



In [ ]:
testData_with_embeddings = embeddings_pipeline.fit(testData).transform(testData)\
                                                  .select("text","category","sentence_embeddings")
testData_with_embeddings.show(2,truncate=60)

+------------------------------------------------------------+----------------+------------------------------------------------------------+
|                                                        text|        category|                                         sentence_embeddings|
+------------------------------------------------------------+----------------+------------------------------------------------------------+
| ADMITTING DIAGNOSIS: Gastrointestinal bleed. HISTORY OF ...|Gastroenterology|[{sentence_embeddings, 0, 3978,  ADMITTING DIAGNOSIS: Gas...|
| CHIEF COMPLAINT: Dysphagia and hematemesis while vomitin...|Gastroenterology|[{sentence_embeddings, 0, 6515,  CHIEF COMPLAINT: Dysphag...|
+------------------------------------------------------------+----------------+------------------------------------------------------------+
only showing top 2 rows



In [ ]:
log_folder="Mt_logs_healthcare_200d"

### ClassifierDL

In [ ]:
classifier_dl = nlp.ClassifierDLApproach()\
        .setInputCols(["sentence_embeddings"])\
        .setOutputCol("class")\
        .setLabelColumn("category")\
        .setBatchSize(8)\
        .setMaxEpochs(50)\
        .setLr(0.005)\
        .setDropout(0.1)\
        .setEnableOutputLogs(True)\
        .setOutputLogsPath(log_folder)

classifier_dl_pipeline = nlp.Pipeline(stages=[classifier_dl])

In [ ]:
clfDL_model_hc200 = classifier_dl_pipeline.fit(trainingData_with_embeddings)

In [ ]:
preds = clfDL_model_hc200.transform(testData_with_embeddings)

preds_df = preds.select("category","text","class.result").toPandas()
preds_df["result"] = preds_df["result"].apply(lambda x : x[0])

print (classification_report(preds_df["category"], preds_df["result"]))

                  precision    recall  f1-score   support

Gastroenterology       0.00      0.00      0.00        25
       Neurology       0.00      0.00      0.00        22
      Orthopedic       0.34      1.00      0.51        35
         Urology       0.00      0.00      0.00        20

        accuracy                           0.34       102
       macro avg       0.09      0.25      0.13       102
    weighted avg       0.12      0.34      0.18       102



In [ ]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["ClassifierDL_200d"] = [res["macro avg"]["f1-score"], res["weighted avg"]["f1-score"], res["accuracy"]]

### MultiClassifierDL



In [ ]:
# MultiClassifierDL accepts list of strings as input label. So we convert label column to array type.
trainingData_with_embeddings = trainingData_with_embeddings.withColumn("category_array", F.array(F.col("category")))
testData_with_embeddings = testData_with_embeddings.withColumn("category_array", F.array(F.col("category")))

In [ ]:
multiClassifier =nlp.MultiClassifierDLApproach()\
  .setInputCols("sentence_embeddings")\
  .setOutputCol("prediction")\
  .setLabelColumn("category_array")\
  .setBatchSize(16)\
  .setMaxEpochs(20)\
  .setLr(9e-3)\
  .setThreshold(0.2)\
  .setShufflePerEpoch(False)\
  .setEnableOutputLogs(True)\
  .setOutputLogsPath(log_folder)\
#   .setValidationSplit(0.1)

multiClassifier_pipeline = nlp.Pipeline(
    stages = [
        multiClassifier
    ])

In [ ]:
multiClassifier_model_hc200 = multiClassifier_pipeline.fit(trainingData_with_embeddings)

In [ ]:
preds = multiClassifier_model_hc200.transform(testData_with_embeddings)

In [ ]:
preds_df = preds.select("category","text","prediction.result","prediction.metadata").toPandas()
preds_df.result.apply(lambda x: str(x) ).value_counts()

['Orthopedic']                        32
['Gastroenterology']                  19
['Urology']                           18
['Neurology']                         13
['Neurology', 'Orthopedic']           11
[]                                     4
['Orthopedic', 'Gastroenterology']     3
['Urology', 'Orthopedic']              1
['Urology', 'Gastroenterology']        1
Name: result, dtype: int64

In [ ]:
# We will get the highest score label as result. you can control the number of zero label results with setThreshold() in the training.
preds_df["scores"] = preds_df.metadata.apply(lambda x: {k:float(v) for k,v in x[0].items()} if len(x)>=1 else "")
preds_df["result"] = preds_df.scores.apply(lambda x: max(x, key=x.get) if len(x)>=1 else "")
preds_df = preds_df[(preds_df["result"]!="")]
preds_df["result"].value_counts()

Orthopedic          38
Gastroenterology    22
Urology             20
Neurology           18
Name: result, dtype: int64

In [ ]:
print (classification_report(preds_df["category"], preds_df["result"]))

                  precision    recall  f1-score   support

Gastroenterology       1.00      0.92      0.96        24
       Neurology       0.89      0.80      0.84        20
      Orthopedic       0.87      0.94      0.90        35
         Urology       0.90      0.95      0.92        19

        accuracy                           0.91        98
       macro avg       0.91      0.90      0.91        98
    weighted avg       0.91      0.91      0.91        98



In [ ]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["MultiClassifierDL_200d"] = [res["macro avg"]["f1-score"], res["weighted avg"]["f1-score"], res["accuracy"]]

### Generic Classifier

In [ ]:
# !pip install -q tensorflow==2.7.0 tensorflow_addons

In [ ]:
from sparknlp_jsl.annotator import TFGraphBuilder

graph_folder = "gc_graph"

gc_graph_builder =medical.TFGraphBuilder()\
    .setModelName("generic_classifier")\
    .setInputCols(["features"])\
    .setLabelColumn("category")\
    .setHiddenLayers([300,200, 50])\
    .setHiddenAct("tanh")\
    .setHiddenActL2(True)\
    .setBatchNorm(True)\
    .setGraphFolder(graph_folder)\
    .setGraphFile("gcf_graph_200d.pb")

In [ ]:
features_asm =medical.FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("features")

gen_clf =medical.GenericClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("features")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/gcf_graph_200d.pb")\
    .setEpochsNumber(30)\
    .setBatchSize(32)\
    .setLearningRate(0.001)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.3)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = nlp.Pipeline(stages=[
    features_asm,
    gc_graph_builder,
    gen_clf])


In [ ]:
generic_model_hc200 = clf_Pipeline.fit(trainingData_with_embeddings)

TF Graph Builder configuration:
Model name: generic_classifier
Graph folder: gc_graph
Graph file name: gcf_graph_200d.pb
Build params: {'input_dim': 200, 'output_dim': 4, 'hidden_layers': [300, 200, 50], 'hidden_act': 'tanh', 'hidden_act_l2': True, 'batch_norm': True}
generic_classifier graph exported to gc_graph/gcf_graph_200d.pb


In [ ]:
pred_df = generic_model_hc200.transform(testData_with_embeddings)

preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

                  precision    recall  f1-score   support

Gastroenterology       0.91      0.84      0.87        25
       Neurology       0.89      0.77      0.83        22
      Orthopedic       0.85      0.94      0.89        35
         Urology       0.86      0.90      0.88        20

        accuracy                           0.87       102
       macro avg       0.88      0.86      0.87       102
    weighted avg       0.88      0.87      0.87       102



In [ ]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["GenericClassifier_200d"] = [res["macro avg"]["f1-score"], res["weighted avg"]["f1-score"], res["accuracy"]]

### GenericLogRegClassifier

In [ ]:
graph_folder = "gc_graph"

gc_logreg_graph_builder = medical.TFGraphBuilder()\
    .setModelName("logreg_classifier")\
    .setInputCols(["feature_vector"]) \
    .setLabelColumn("category")\
    .setGraphFolder(graph_folder)\
    .setGraphFile("log_reg_graph.pb")

GenericLogRegClassifier needs outputs from FeaturesAssembler. The FeaturesAssembler is used to collect features from different columns or an embeddings column.

The GenericLogRegClassifier takes FEATURE_VECTOR annotations as input, classifies them and outputs CATEGORY annotations.

In [ ]:
features_asm = medical.FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("feature_vector")

gen_clf = medical.GenericLogRegClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("feature_vector")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/log_reg_graph.pb")\
    .setEpochsNumber(45)\
    .setBatchSize(32)\
    .setLearningRate(0.02)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.1)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = nlp.Pipeline(stages=[
    features_asm,
    gc_logreg_graph_builder,
    gen_clf])


In [ ]:
generic_model_hc200 = clf_Pipeline.fit(trainingData_with_embeddings)

TF Graph Builder configuration:
Model name: logreg_classifier
Graph folder: gc_graph
Graph file name: log_reg_graph.pb
Build params: {'input_dim': 200, 'output_dim': 4, 'hidden_layers': [], 'output_act': 'sigmoid'}
logreg_classifier graph exported to gc_graph/log_reg_graph.pb


In [ ]:
pred_df = generic_model_hc200.transform(testData_with_embeddings)

In [ ]:
preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

                  precision    recall  f1-score   support

Gastroenterology       0.88      0.84      0.86        25
       Neurology       0.81      0.77      0.79        22
      Orthopedic       0.86      0.91      0.89        35
         Urology       0.80      0.80      0.80        20

        accuracy                           0.84       102
       macro avg       0.84      0.83      0.83       102
    weighted avg       0.84      0.84      0.84       102



In [ ]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["GenericLogReg_200d"] = [res["macro avg"]["f1-score"], res["weighted avg"]["f1-score"], res["accuracy"]]

### GenericSVMClassifier

In [ ]:
graph_folder = "gc_graph"

gc_svm_graph_builder = medical.TFGraphBuilder()\
    .setModelName("svm_classifier")\
    .setInputCols(["feature_vector"]) \
    .setLabelColumn("category")\
    .setGraphFolder(graph_folder)\
    .setGraphFile("svm_graph.pb")

GenericSVMClassifier needs outputs from FeaturesAssembler. The FeaturesAssembler is used to collect features from different columns or an embeddings column.

The GenericSVMClassifier takes FEATURE_VECTOR annotations as input, classifies them and outputs CATEGORY annotations.

In [ ]:
features_asm = medical.FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("feature_vector")

gen_clf = medical.GenericSVMClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("feature_vector")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/svm_graph.pb")\
    .setEpochsNumber(45)\
    .setBatchSize(32)\
    .setLearningRate(0.02)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.1)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = nlp.Pipeline(stages=[
    features_asm,
    gc_svm_graph_builder,
    gen_clf])


In [ ]:
generic_model_hc200 = clf_Pipeline.fit(trainingData_with_embeddings)

TF Graph Builder configuration:
Model name: svm_classifier
Graph folder: gc_graph
Graph file name: svm_graph.pb
Build params: {'input_dim': 200, 'output_dim': 4, 'hidden_layers': [], 'output_act': 'sigmoid', 'loss_func': 'hinge'}
svm_classifier graph exported to gc_graph/svm_graph.pb


In [ ]:
pred_df = generic_model_hc200.transform(testData_with_embeddings)

In [ ]:
preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

                  precision    recall  f1-score   support

Gastroenterology       0.81      0.84      0.82        25
       Neurology       0.86      0.82      0.84        22
      Orthopedic       0.89      0.91      0.90        35
         Urology       0.84      0.80      0.82        20

        accuracy                           0.85       102
       macro avg       0.85      0.84      0.85       102
    weighted avg       0.85      0.85      0.85       102



In [ ]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["GenericSVM_200d"] = [res["macro avg"]["f1-score"], res["weighted avg"]["f1-score"], res["accuracy"]]

## Bert Sentence Embeddings (sbiobert_base_cased_mli)

We will extract [sbiobert_base_cased_mli](https://nlp.johnsnowlabs.com/2020/11/27/sbiobert_base_cased_mli_en.html) embeddings which has 768 dimension output and use this embeddings in the model training.

In [ ]:
document_assembler = nlp.DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

bert_sent = nlp.BertSentenceEmbeddings.pretrained("sbiobert_base_cased_mli", 'en','clinical/models')\
    .setInputCols(["document"])\
    .setOutputCol("sentence_embeddings")

embeddings_pipeline = nlp.Pipeline(
    stages = [document_assembler,
              bert_sent])

sbiobert_base_cased_mli download started this may take some time.
Approximate size to download 384.3 MB
[OK!]


In [ ]:
trainingData_with_embeddings = embeddings_pipeline.fit(trainingData).transform(trainingData)\
                                                  .select("text","category","sentence_embeddings")

trainingData_with_embeddings.show(2,truncate=60)

+------------------------------------------------------------+----------------+------------------------------------------------------------+
|                                                        text|        category|                                         sentence_embeddings|
+------------------------------------------------------------+----------------+------------------------------------------------------------+
| ADMISSION DIAGNOSIS: Symptomatic cholelithiasis. DISCHAR...|Gastroenterology|[{sentence_embeddings, 0, 2228,  ADMISSION DIAGNOSIS: Sym...|
| ADMITTING DIAGNOSES: Hiatal hernia, gastroesophageal ref...|Gastroenterology|[{sentence_embeddings, 0, 3237,  ADMITTING DIAGNOSES: Hia...|
+------------------------------------------------------------+----------------+------------------------------------------------------------+
only showing top 2 rows



In [ ]:
testData_with_embeddings = embeddings_pipeline.fit(testData).transform(testData)\
                                                  .select("text","category","sentence_embeddings")

testData_with_embeddings.show(2,truncate=60)

+------------------------------------------------------------+----------------+------------------------------------------------------------+
|                                                        text|        category|                                         sentence_embeddings|
+------------------------------------------------------------+----------------+------------------------------------------------------------+
| ADMITTING DIAGNOSIS: Gastrointestinal bleed. HISTORY OF ...|Gastroenterology|[{sentence_embeddings, 0, 3978,  ADMITTING DIAGNOSIS: Gas...|
| CHIEF COMPLAINT: Dysphagia and hematemesis while vomitin...|Gastroenterology|[{sentence_embeddings, 0, 6515,  CHIEF COMPLAINT: Dysphag...|
+------------------------------------------------------------+----------------+------------------------------------------------------------+
only showing top 2 rows



In [ ]:
log_folder="Mt_logs_bert"

### ClassifierDL

In [ ]:
classifier_dl = nlp.ClassifierDLApproach()\
        .setInputCols(["sentence_embeddings"])\
        .setOutputCol("class")\
        .setLabelColumn("category")\
        .setBatchSize(8)\
        .setMaxEpochs(30)\
        .setLr(0.002)\
        .setDropout(0.1)\
        .setEnableOutputLogs(True)\
        .setOutputLogsPath(log_folder)

classifier_dl_pipeline = nlp.Pipeline(stages=[classifier_dl])

In [ ]:
clfDL_model_bert = classifier_dl_pipeline.fit(trainingData_with_embeddings)

In [ ]:
preds = clfDL_model_bert.transform(testData_with_embeddings)

preds_df = preds.select("category","text","class.result").toPandas()
preds_df["result"] = preds_df["result"].apply(lambda x : x[0])

print (classification_report(preds_df["category"], preds_df["result"]))

                  precision    recall  f1-score   support

Gastroenterology       0.92      0.88      0.90        25
       Neurology       0.83      0.86      0.84        22
      Orthopedic       0.88      0.86      0.87        35
         Urology       0.90      0.95      0.93        20

        accuracy                           0.88       102
       macro avg       0.88      0.89      0.88       102
    weighted avg       0.88      0.88      0.88       102



In [ ]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["ClassifierDL_bert"] = [res["macro avg"]["f1-score"], res["weighted avg"]["f1-score"], res["accuracy"]]

### MultiClassifierDL

In [ ]:
# MultiClassifierDL accepts list of strings as input label. So we convert label column to array type.
trainingData_with_embeddings = trainingData_with_embeddings.withColumn("category_array", F.array(F.col("category")))
testData_with_embeddings = testData_with_embeddings.withColumn("category_array", F.array(F.col("category")))

In [ ]:
multiClassifier =nlp.MultiClassifierDLApproach()\
  .setInputCols("sentence_embeddings")\
  .setOutputCol("prediction")\
  .setLabelColumn("category_array")\
  .setBatchSize(8)\
  .setMaxEpochs(20)\
  .setLr(9e-3)\
  .setThreshold(0.5)\
  .setShufflePerEpoch(False)\
  .setEnableOutputLogs(True)\
  .setOutputLogsPath(log_folder)\
#   .setValidationSplit(0.1)

multiClassifier_pipeline = nlp.Pipeline(
    stages = [
        multiClassifier
    ])

In [ ]:
multiClassifier_model_bert = multiClassifier_pipeline.fit(trainingData_with_embeddings)

In [ ]:
preds = multiClassifier_model_bert.transform(testData_with_embeddings)

In [ ]:
preds_df = preds.select("category","text","prediction.result","prediction.metadata").toPandas()
preds_df.result.apply(lambda x: str(x) ).value_counts()

['Orthopedic']                 32
['Gastroenterology']           25
['Neurology']                  22
['Urology']                    21
[]                              1
['Neurology', 'Orthopedic']     1
Name: result, dtype: int64

In [ ]:
# We will get the highest score label as result. You can control the number of zero label results with setThreshold() in the training.
preds_df["scores"] = preds_df.metadata.apply(lambda x: {k:float(v) for k,v in x[0].items()} if len(x)>=1 else "")
preds_df["result"] = preds_df.scores.apply(lambda x: max(x, key=x.get) if len(x)>=1 else "")
preds_df = preds_df[(preds_df["result"]!="")]
preds_df["result"].value_counts()

Orthopedic          32
Gastroenterology    25
Neurology           23
Urology             21
Name: result, dtype: int64

In [ ]:
print (classification_report(preds_df["category"], preds_df["result"]))

                  precision    recall  f1-score   support

Gastroenterology       0.92      0.92      0.92        25
       Neurology       0.78      0.86      0.82        21
      Orthopedic       0.91      0.83      0.87        35
         Urology       0.86      0.90      0.88        20

        accuracy                           0.87       101
       macro avg       0.87      0.88      0.87       101
    weighted avg       0.87      0.87      0.87       101



In [ ]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["MultiClassifierDL_bert"] = [res["macro avg"]["f1-score"], res["weighted avg"]["f1-score"], res["accuracy"]]

### Generic Classifier

In [ ]:
# !pip install -q tensorflow==2.7.0 tensorflow_addons

In [ ]:
from sparknlp_jsl.annotator import TFGraphBuilder

graph_folder = "gc_graph"

gc_graph_builder =medical.TFGraphBuilder()\
    .setModelName("generic_classifier")\
    .setInputCols(["features"])\
    .setLabelColumn("category")\
    .setHiddenLayers([300,200, 50])\
    .setHiddenAct("tanh")\
    .setHiddenActL2(True)\
    .setBatchNorm(True)\
    .setGraphFolder(graph_folder)\
    .setGraphFile("gcf_graph_bert.pb")

In [ ]:
features_asm =medical.FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("features")

gen_clf =medical.GenericClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("features")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/gcf_graph_bert.pb")\
    .setEpochsNumber(25)\
    .setBatchSize(16)\
    .setLearningRate(0.002)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.2)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = nlp.Pipeline(stages=[
    features_asm,
    gc_graph_builder,
    gen_clf])


In [ ]:
generic_model_bert = clf_Pipeline.fit(trainingData_with_embeddings)

TF Graph Builder configuration:
Model name: generic_classifier
Graph folder: gc_graph
Graph file name: gcf_graph_bert.pb
Build params: {'input_dim': 768, 'output_dim': 4, 'hidden_layers': [300, 200, 50], 'hidden_act': 'tanh', 'hidden_act_l2': True, 'batch_norm': True}
generic_classifier graph exported to gc_graph/gcf_graph_bert.pb


In [ ]:
pred_df = generic_model_bert.transform(testData_with_embeddings)

preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

                  precision    recall  f1-score   support

Gastroenterology       1.00      0.88      0.94        25
       Neurology       0.73      0.86      0.79        22
      Orthopedic       0.88      0.83      0.85        35
         Urology       0.86      0.90      0.88        20

        accuracy                           0.86       102
       macro avg       0.87      0.87      0.86       102
    weighted avg       0.87      0.86      0.87       102



In [ ]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["GenericClassifier_bert"] = [res["macro avg"]["f1-score"], res["weighted avg"]["f1-score"], res["accuracy"]]

### GenericLogRegClassifier

In [ ]:
graph_folder = "gc_graph"

gc_logreg_graph_builder = medical.TFGraphBuilder()\
    .setModelName("logreg_classifier")\
    .setInputCols(["feature_vector"]) \
    .setLabelColumn("category")\
    .setGraphFolder(graph_folder)\
    .setGraphFile("log_reg_graph.pb")

GenericLogRegClassifier needs outputs from FeaturesAssembler. The FeaturesAssembler is used to collect features from different columns or an embeddings column.

The GenericLogRegClassifier takes FEATURE_VECTOR annotations as input, classifies them and outputs CATEGORY annotations.

In [ ]:
features_asm = medical.FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("feature_vector")

gen_clf = medical.GenericLogRegClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("feature_vector")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/log_reg_graph.pb")\
    .setEpochsNumber(30)\
    .setBatchSize(64)\
    .setLearningRate(0.004)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.1)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = nlp.Pipeline(stages=[
    features_asm,
    gc_logreg_graph_builder,
    gen_clf])


In [ ]:
generic_model_bert = clf_Pipeline.fit(trainingData_with_embeddings)

TF Graph Builder configuration:
Model name: logreg_classifier
Graph folder: gc_graph
Graph file name: log_reg_graph.pb
Build params: {'input_dim': 768, 'output_dim': 4, 'hidden_layers': [], 'output_act': 'sigmoid'}
logreg_classifier graph exported to gc_graph/log_reg_graph.pb


In [ ]:
pred_df = generic_model_bert.transform(testData_with_embeddings)

In [ ]:
preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

                  precision    recall  f1-score   support

Gastroenterology       0.96      0.92      0.94        25
       Neurology       0.83      0.86      0.84        22
      Orthopedic       0.91      0.86      0.88        35
         Urology       0.86      0.95      0.90        20

        accuracy                           0.89       102
       macro avg       0.89      0.90      0.89       102
    weighted avg       0.89      0.89      0.89       102



In [ ]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["GenericCLogReg_bert"] = [res["macro avg"]["f1-score"], res["weighted avg"]["f1-score"], res["accuracy"]]

### GenericSVMClassifier

In [ ]:
graph_folder = "gc_graph"

gc_svm_graph_builder = medical.TFGraphBuilder()\
    .setModelName("svm_classifier")\
    .setInputCols(["feature_vector"]) \
    .setLabelColumn("category")\
    .setGraphFolder(graph_folder)\
    .setGraphFile("svm_graph.pb")

GenericSVMClassifier needs outputs from FeaturesAssembler. The FeaturesAssembler is used to collect features from different columns or an embeddings column.

The GenericSVMClassifier takes FEATURE_VECTOR annotations as input, classifies them and outputs CATEGORY annotations.

In [ ]:
features_asm = medical.FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("feature_vector")

gen_clf = medical.GenericSVMClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("feature_vector")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/svm_graph.pb")\
    .setEpochsNumber(30)\
    .setBatchSize(64)\
    .setLearningRate(0.004)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.1)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = nlp.Pipeline(stages=[
    features_asm,
    gc_svm_graph_builder,
    gen_clf])


In [ ]:
generic_model_bert = clf_Pipeline.fit(trainingData_with_embeddings)

TF Graph Builder configuration:
Model name: svm_classifier
Graph folder: gc_graph
Graph file name: svm_graph.pb
Build params: {'input_dim': 768, 'output_dim': 4, 'hidden_layers': [], 'output_act': 'sigmoid', 'loss_func': 'hinge'}
svm_classifier graph exported to gc_graph/svm_graph.pb


In [ ]:
pred_df = generic_model_bert.transform(testData_with_embeddings)

In [ ]:
preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

                  precision    recall  f1-score   support

Gastroenterology       0.88      0.92      0.90        25
       Neurology       0.81      0.95      0.88        22
      Orthopedic       0.97      0.86      0.91        35
         Urology       0.89      0.85      0.87        20

        accuracy                           0.89       102
       macro avg       0.89      0.90      0.89       102
    weighted avg       0.90      0.89      0.89       102



In [ ]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["GenericCSVM_bert"] = [res["macro avg"]["f1-score"], res["weighted avg"]["f1-score"], res["accuracy"]]

## DocumentLogRegClassifier

In [ ]:
document_assembler = nlp.DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

tokenizer = nlp.Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

logreg = medical.DocumentLogRegClassifierApproach()\
    .setInputCols("token")\
    .setLabelCol("category")\
    .setOutputCol("prediction")\
    .setMaxIter(10)\
    .setTol(1e-6)

clf_Pipeline = nlp.Pipeline(stages=[document_assembler,
                                tokenizer,
                                logreg])

doclogreg_model = clf_Pipeline.fit(trainingData)

In [ ]:
pred_df = doclogreg_model.transform(testData)

preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

                  precision    recall  f1-score   support

Gastroenterology       0.88      0.88      0.88        25
       Neurology       0.81      0.77      0.79        22
      Orthopedic       0.86      0.89      0.87        35
         Urology       0.90      0.90      0.90        20

        accuracy                           0.86       102
       macro avg       0.86      0.86      0.86       102
    weighted avg       0.86      0.86      0.86       102



In [ ]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["DocumentLogRegClassifier"] = [res["macro avg"]["f1-score"], res["weighted avg"]["f1-score"], res["accuracy"]]

## Comparision of All Models' Result in Mtsamples Dataset

Here is the all results of classifier models. Macro f1, weighted f1 and accuracy scores are presented.

In [ ]:
results_df

,macro-f1-score,weighted-f1-score,accuracy
ClassifierDL_100d,0.708342,0.738724,0.745098
MultiClassifierDL_100d,0.884165,0.885508,0.886598
GenericClassifier_100d,0.879519,0.883000,0.882353
GenericLogReg_100d,0.859291,0.862404,0.862745
GenericSVM_100d,0.877119,0.882451,0.882353
ClassifierDL_200d,0.127737,0.175326,0.343137
MultiClassifierDL_200d,0.906453,0.907969,0.908163
GenericClassifier_200d,0.868552,0.871530,0.872549
GenericLogReg_200d,0.834182,0.842500,0.843137
GenericSVM_200d,0.845665,0.852612,0.852941
